
# MIMIC‑IV Hypercapnia Cohort — **ICD ∪ Physiologic Thresholds** (BigQuery)

TODO: 

[ ] want to add comorbidity data (any codes)

[ ] want to add ED-rendered diagnoses

[ ] want to see if we can pull info on potential causative diagnoses rendered when making this data-set


**Goal:** Build an admissions‑level tabular dataset that **enrolls** any hospital admission (`hadm_id`) meeting **_any_** of:

1. **ICD** codes for hypercapnic respiratory failure (legacy cohort).
2. **Any arterial blood gas** (LAB or POC) with **PaCO₂ ≥ 45.0 mmHg** anywhere during the episode.
3. **Any venous blood gas** (LAB or POC) with **PaCO₂ ≥ 50.0 mmHg** anywhere during the episode.

Then, keep all downstream columns/logic from the current workflow:
- Per‑code ICD indicators and an `any_hypercap_icd` flag.
- Robust extraction of **first ABG** and **first VBG** (across LAB + POC) with standardized units (mmHg) and pairing logic.
- Demographics/outcomes, NIH/OMB race & ethnicity, ED triage + first ED vitals, ICU meta (first stay + LOS), ventilation flags.
- Sanity checks.

> **Assumptions**
> - You already configured BigQuery auth (`gcloud auth application-default login`) and `.env` variables as in the previous notebook.
> - The PhysioNet hosting project is `physionet-data`.
> - Datasets exist (e.g., `mimiciv_3_1_hosp`, `mimiciv_3_1_icu`, and an ED dataset such as `mimiciv_ed`). This notebook auto-detects the ED dataset.


**Execution timing note:** To capture per-cell runtimes, enable cell execution timing in VS Code (Notebook: Show Cell Execution Time) or enable ExecuteTime in Jupyter. Then re-run and save to allow runtime summaries.


## Pipeline stages (readable map)
1. **Stage A — Config & helpers**: dataset IDs, thresholds, shared helper functions.
2. **Stage B — Cohort spine**: ICD + gas thresholds → `hadm_list` / `ed_stay_list`.
3. **Stage C — Bulk pulls**: one query per table where possible.
4. **Stage D — Transforms**: panels, flags, timing, comorbidities, vitals.
5. **Stage E — QA checks**: deterministic assertions and range checks.
6. **Stage F — Outputs**: parquet + Excel exports.


## SQL registry (centralized query templates)
All BigQuery SQL is defined in one place below, then referenced by name in subsequent cells.


In [1]:
import sys
print(sys.executable)

# Central SQL registry (define all query templates here)
SQL = {}

def sql(name: str) -> str:
    if name not in SQL:
        raise KeyError(f"SQL template not found: {name}")
    return SQL[name]

# SQL templates (populated below in-place to keep notebook linear)
# Names: admit_sql, co2_thresholds_sql, cohort_icd_sql, counts_sql, demo_sql, ditems_sql, ed_counts_sql, ed_first_vitals_sql, ed_spine_sql, ed_to_icu_sql, ed_triage_sql, ed_vitals_sql, icd_sql, icu_meta_sql, icu_sql, labitems_sql, labs_sql, vent_chart_sql, vent_sql


/Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/.venv/bin/python


**Rationale:** Define a reproducible, admission-level cohort that captures hypercapnia using complementary diagnostic (ICD) and physiologic (blood gas) criteria.


# MIMIC‑IV on BigQuery

## Environment Bootstrap & Smoke Test

Purpose: make a clean, reproducible start on a new machine.

Outcome: verify auth, project config, and dataset access; provide a reusable BigQuery runner for the build notebook.

**Rationale:** Establish the BigQuery environment and dataset configuration so queries are consistent and reproducible across runs.


## 0. Prerequisites (one-time)

**Accounts & access**
- PhysioNet access to MIMIC-IV on BigQuery; in BigQuery Console star project `physionet-data`.
- A Google Cloud **Project ID** with BigQuery API enabled (this is your **billing** project).

**CLI & environment**
- Google Cloud SDK (gcloud) installed and on PATH.
- Python environment created with `uv` (see README) and Jupyter kernel selected.
- A project-local **`.env`** with the variables below.

**.env variables**
```ini
MIMIC_BACKEND=bigquery
WORK_PROJECT=<your-billing-project-id>
BQ_PHYSIONET_PROJECT=physionet-data
BQ_DATASET_HOSP=mimiciv_3_1_hosp
BQ_DATASET_ICU=mimiciv_3_1_icu
BQ_DATASET_ED=mimiciv_ed
WORK_DIR=/path/to/Hypercap-CC-NLP
# GOOGLE_APPLICATION_CREDENTIALS=/path/to/service-account.json
```

**Command line quickstart**
```bash
brew install --cask google-cloud-sdk
gcloud init
gcloud auth application-default login
gcloud services enable bigquery.googleapis.com --project <your-billing-project-id>
ls -l ~/.config/gcloud/application_default_credentials.json
```


**Rationale:** Verify access and credentials up front to prevent silent failures later in the pipeline.


In [2]:
# --- Imports & environment
import os, re, json, math, textwrap
from pathlib import Path

import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv

load_dotenv()

WORK_DIR = Path(os.getenv("WORK_DIR", Path.cwd())).expanduser().resolve()
DATA_DIR = WORK_DIR / "MIMIC tabular data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ---- Backend selection (we use BigQuery)
BACKEND = os.getenv("MIMIC_BACKEND", "bigquery").strip().lower()
assert BACKEND == "bigquery", "This notebook is BigQuery-specific."

WORK_PROJECT = os.getenv("WORK_PROJECT", "").strip()  # your billing project
PHYS = os.getenv("BQ_PHYSIONET_PROJECT", "physionet-data").strip()  # hosting project (read-only)

# Default to MIMIC-IV v3.1 datasets
HOSP = os.getenv("BQ_DATASET_HOSP", "mimiciv_3_1_hosp").strip()
ICU  = os.getenv("BQ_DATASET_ICU",  "mimiciv_3_1_icu").strip()
ED   = os.getenv("BQ_DATASET_ED",   "").strip()  # may be empty; we will auto-detect

# BigQuery client
client = bigquery.Client(project=WORK_PROJECT)

print("Project:", WORK_PROJECT)
print("PhysioNet host:", PHYS)
print("HOSP:", HOSP, "ICU:", ICU, "ED (pref):", ED)
print("WORK_DIR:", WORK_DIR)



Project: mimic-hypercapnia
PhysioNet host: physionet-data
HOSP: mimiciv_3_1_hosp ICU: mimiciv_3_1_icu ED (pref): mimiciv_ed
WORK_DIR: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP


In [3]:
# --- Helper: run SQL with optional named parameters
def run_sql_bq(sql: str, params: dict | None = None) -> pd.DataFrame:
    job_config = bigquery.QueryJobConfig()
    if params:
        bq_params = []
        for k, v in params.items():
            if isinstance(v, (list, tuple, np.ndarray, pd.Series)):
                # BigQuery ARRAY<INT64> if all ints; else ARRAY<STRING>
                v_list = list(v)
                if all(isinstance(x, (int, np.integer)) for x in v_list):
                    bq_params.append(bigquery.ArrayQueryParameter(k, "INT64", list(map(int, v_list))))
                else:
                    bq_params.append(bigquery.ArrayQueryParameter(k, "STRING", list(map(str, v_list))))
            else:
                # scalar
                if isinstance(v, (int, np.integer)):
                    bq_params.append(bigquery.ScalarQueryParameter(k, "INT64", int(v)))
                elif isinstance(v, float):
                    bq_params.append(bigquery.ScalarQueryParameter(k, "FLOAT64", float(v)))
                else:
                    bq_params.append(bigquery.ScalarQueryParameter(k, "STRING", str(v)))
        job_config.query_parameters = bq_params
    job = client.query(sql, job_config=job_config)
    try:
        return job.result().to_dataframe(create_bqstorage_client=True)
    except TypeError:
        return job.result().to_dataframe()

# --- Helper: test if a fully-qualified table exists and is accessible
def table_exists(fqtn: str) -> bool:
    try:
        _ = run_sql_bq(f"SELECT 1 FROM `{fqtn}` LIMIT 1")
        return True
    except Exception:
        return False

# --- Auto-detect ED dataset if not set
if not ED:
    candidates = [
        f"{PHYS}.mimiciv_ed.edstays",
        f"{PHYS}.mimiciv_3_1_ed.edstays",
    ]
    for cand in candidates:
        if table_exists(cand):
            ED = cand.split(".")[1]  # dataset
            break
if not ED:
    raise RuntimeError("No accessible ED dataset found. Request access to MIMIC-IV-ED in PhysioNet (BigQuery).")

print("Using ED dataset:", ED)



Using ED dataset: mimiciv_ed


In [4]:
# --- Helper utilities for reproducibility and safe joins
def require_cols(df: pd.DataFrame, cols: list[str], name: str) -> None:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise KeyError(f"{name} missing columns: {missing}")

def assert_unique(df: pd.DataFrame, key: str, name: str) -> None:
    if df[key].duplicated().any():
        n = int(df[key].duplicated().sum())
        raise ValueError(f"{name} has {n} duplicate {key} values")

def safe_merge(left: pd.DataFrame, right: pd.DataFrame, on: list[str] | str, how: str, name: str) -> pd.DataFrame:
    # guard against accidental duplicate columns
    overlap = set(left.columns) & set(right.columns)
    if isinstance(on, str):
        on_cols = {on}
    else:
        on_cols = set(on)
    overlap = overlap - on_cols
    if overlap:
        raise ValueError(f"{name} merge would duplicate columns: {sorted(overlap)}")
    return left.merge(right, on=on, how=how)

def check_ranges(df: pd.DataFrame, ranges: dict[str, tuple[float, float]]) -> pd.DataFrame:
    rows = []
    for col, (lo, hi) in ranges.items():
        if col not in df.columns:
            continue
        bad = df[col].notna() & ((df[col] < lo) | (df[col] > hi))
        rows.append({"col": col, "n_bad": int(bad.sum())})
    return pd.DataFrame(rows)



## 1) ICD cohort flags (hypercapnic respiratory failure)

**Rationale:** Capture diagnosis-based hypercapnia from ED and hospital discharge codes to define a broad, clinically recognized cohort.


In [5]:
# Target ICD codes (dotless, uppercase)
ICD10_CODES = ['J9602','J9612','J9622','J9692','E662']
ICD9_CODES  = ['27803']

SQL["cohort_icd_sql"] = f"""
-- ICD-based cohort flags per admission
WITH target_codes AS (
  SELECT 'J9602' AS code, 10 AS ver UNION ALL
  SELECT 'J9612', 10 UNION ALL
  SELECT 'J9622', 10 UNION ALL
  SELECT 'J9692', 10 UNION ALL
  SELECT 'E662',  10 UNION ALL
  SELECT '27803', 9
),

-- Hospital ICDs restricted to target codes
hosp_dx AS (
  SELECT
    d.subject_id,
    d.hadm_id,
    UPPER(REPLACE(d.icd_code, '.', '')) AS code_norm,
    d.icd_version
  FROM `{PHYS}.{HOSP}.diagnoses_icd` d
  JOIN target_codes t
    ON t.ver = d.icd_version AND t.code = UPPER(REPLACE(d.icd_code, '.', ''))
  WHERE d.hadm_id IS NOT NULL
),

-- Hospital flags per admission
hosp_flags AS (
  SELECT
    subject_id, hadm_id,
    MAX(IF(icd_version=10 AND code_norm='J9602',1,0)) AS ICD10_J9602,
    MAX(IF(icd_version=10 AND code_norm='J9612',1,0)) AS ICD10_J9612,
    MAX(IF(icd_version=10 AND code_norm='J9622',1,0)) AS ICD10_J9622,
    MAX(IF(icd_version=10 AND code_norm='J9692',1,0)) AS ICD10_J9692,
    MAX(IF(icd_version=10 AND code_norm='E662', 1,0)) AS ICD10_E662,
    MAX(IF(icd_version=9  AND code_norm='27803',1,0)) AS ICD9_27803
  FROM hosp_dx
  GROUP BY subject_id, hadm_id
),

-- ED ICDs restricted to target codes (map to hadm via edstays)
ed_dx AS (
  SELECT
    s.subject_id,
    s.hadm_id,
    s.stay_id,
    s.intime AS ed_intime,
    UPPER(REPLACE(d.icd_code, '.', '')) AS code_norm,
    d.icd_version
  FROM `{PHYS}.{ED}.diagnosis` d
  JOIN `{PHYS}.{ED}.edstays` s
    ON s.subject_id = d.subject_id AND s.stay_id = d.stay_id
  JOIN target_codes t
    ON t.ver = d.icd_version AND t.code = UPPER(REPLACE(d.icd_code, '.', ''))
  WHERE s.hadm_id IS NOT NULL
),

-- ED flags per ED stay (so we can both: OR flags across stays and also pick earliest stay_id)
ed_flags_by_stay AS (
  SELECT
    subject_id, hadm_id, stay_id, MIN(ed_intime) AS ed_intime,
    MAX(IF(icd_version=10 AND code_norm='J9602',1,0)) AS ICD10_J9602,
    MAX(IF(icd_version=10 AND code_norm='J9612',1,0)) AS ICD10_J9612,
    MAX(IF(icd_version=10 AND code_norm='J9622',1,0)) AS ICD10_J9622,
    MAX(IF(icd_version=10 AND code_norm='J9692',1,0)) AS ICD10_J9692,
    MAX(IF(icd_version=10 AND code_norm='E662', 1,0)) AS ICD10_E662,
    MAX(IF(icd_version=9  AND code_norm='27803',1,0)) AS ICD9_27803
  FROM ed_dx
  GROUP BY subject_id, hadm_id, stay_id
),

-- OR the ED flags across all ED stays mapped to the same hadm
ed_flags_or AS (
  SELECT
    subject_id, hadm_id,
    MAX(ICD10_J9602) AS ICD10_J9602,
    MAX(ICD10_J9612) AS ICD10_J9612,
    MAX(ICD10_J9622) AS ICD10_J9622,
    MAX(ICD10_J9692) AS ICD10_J9692,
    MAX(ICD10_E662 ) AS ICD10_E662,
    MAX(ICD9_27803) AS ICD9_27803
  FROM ed_flags_by_stay
  GROUP BY subject_id, hadm_id
),

-- Earliest ED stay_id per hadm (NO UNNEST of aggregates; use [OFFSET(0)])
ed_earliest AS (
  SELECT
    subject_id,
    hadm_id,
    (ARRAY_AGG(STRUCT(stay_id, ed_intime) ORDER BY ed_intime LIMIT 1))[OFFSET(0)].stay_id AS stay_id
  FROM ed_flags_by_stay
  GROUP BY subject_id, hadm_id
),

-- Bring flags and earliest stay_id together
ed_by_hadm AS (
  SELECT
    f.subject_id,
    f.hadm_id,
    e.stay_id,
    f.ICD10_J9602,
    f.ICD10_J9612,
    f.ICD10_J9622,
    f.ICD10_J9692,
    f.ICD10_E662,
    f.ICD9_27803
  FROM ed_flags_or f
  LEFT JOIN ed_earliest e
    USING (subject_id, hadm_id)
),

-- Combine ED and hospital flags at the admission level
combined AS (
  SELECT
    COALESCE(h.subject_id, e.subject_id) AS subject_id,
    COALESCE(h.hadm_id,     e.hadm_id)   AS hadm_id,
    GREATEST(IFNULL(h.ICD10_J9602,0), IFNULL(e.ICD10_J9602,0)) AS ICD10_J9602,
    GREATEST(IFNULL(h.ICD10_J9612,0), IFNULL(e.ICD10_J9612,0)) AS ICD10_J9612,
    GREATEST(IFNULL(h.ICD10_J9622,0), IFNULL(e.ICD10_J9622,0)) AS ICD10_J9622,
    GREATEST(IFNULL(h.ICD10_J9692,0), IFNULL(e.ICD10_J9692,0)) AS ICD10_J9692,
    GREATEST(IFNULL(h.ICD10_E662 ,0), IFNULL(e.ICD10_E662 ,0)) AS ICD10_E662,
    GREATEST(IFNULL(h.ICD9_27803,0), IFNULL(e.ICD9_27803,0)) AS ICD9_27803,
    IF((IFNULL(h.ICD10_J9602,0)+IFNULL(h.ICD10_J9612,0)+IFNULL(h.ICD10_J9622,0)+IFNULL(h.ICD10_J9692,0)+IFNULL(h.ICD10_E662,0)+IFNULL(h.ICD9_27803,0)) > 0, 1, 0) AS any_hypercap_icd_hosp,
    IF((IFNULL(e.ICD10_J9602,0)+IFNULL(e.ICD10_J9612,0)+IFNULL(e.ICD10_J9622,0)+IFNULL(e.ICD10_J9692,0)+IFNULL(e.ICD10_E662,0)+IFNULL(e.ICD9_27803,0)) > 0, 1, 0) AS any_hypercap_icd_ed
  FROM hosp_flags h
  FULL OUTER JOIN ed_by_hadm e
    ON h.hadm_id = e.hadm_id
)

SELECT
  subject_id, hadm_id,
  ICD10_J9602, ICD10_J9612, ICD10_J9622, ICD10_J9692, ICD10_E662, ICD9_27803,
  IF((ICD10_J9602+ICD10_J9612+ICD10_J9622+ICD10_J9692+ICD10_E662+ICD9_27803) > 0, 1, 0) AS any_hypercap_icd,
  any_hypercap_icd_hosp,
  any_hypercap_icd_ed,
  CASE
    WHEN any_hypercap_icd_hosp=1 AND any_hypercap_icd_ed=1 THEN 'ED+HOSP'
    WHEN any_hypercap_icd_ed=1 THEN 'ED'
    WHEN any_hypercap_icd_hosp=1 THEN 'HOSP'
    ELSE 'NONE'
  END AS icd_source
FROM combined
"""

cohort_icd = run_sql_bq(sql("cohort_icd_sql"))
print("ICD cohort admissions:", len(cohort_icd))
cohort_icd.head(3)



ICD cohort admissions: 4237


,subject_id,hadm_id,ICD10_J9602,ICD10_J9612,ICD10_J9622,ICD10_J9692,ICD10_E662,ICD9_27803,any_hypercap_icd,any_hypercap_icd_hosp,any_hypercap_icd_ed,icd_source
0,10485425,21207827,0,0,1,0,1,0,1,1,0,HOSP
1,16826447,21830147,0,1,0,0,1,0,1,1,0,HOSP
2,11482582,22869017,0,0,0,0,0,1,1,1,0,HOSP


## 2) Blood gas inclusion thresholds — ANY PaCO₂ meeting cutoffs (LAB + POC)

**Rationale:** Identify physiologic hypercapnia using ABG/VBG thresholds, independent of diagnostic coding.


In [6]:
SQL["co2_thresholds_sql"] = f"""
/* ---- LAB (HOSP) pCO2 across entire dataset ---- */
WITH hosp_cand AS (
  SELECT
    le.hadm_id, le.charttime, le.specimen_id,
    COALESCE(
      CAST(le.valuenum AS FLOAT64),
      SAFE_CAST(REGEXP_EXTRACT(LOWER(le.value), r'(-?\d+(?:\.\d+)?)') AS FLOAT64)
    ) AS val,
    LOWER(REPLACE(COALESCE(le.valueuom,''),' ','')) AS uom_nospace,
    LOWER(di.label) AS lbl,
    LOWER(COALESCE(di.fluid,'')) AS fl
  FROM `{PHYS}.{HOSP}.labevents` le
  JOIN `{PHYS}.{HOSP}.d_labitems` di ON di.itemid = le.itemid
  WHERE (le.valuenum IS NOT NULL OR le.value IS NOT NULL)
    AND (
      LOWER(COALESCE(di.category,'')) LIKE '%blood gas%' OR
      LOWER(di.label) LIKE '%pco2%' OR
      REGEXP_CONTAINS(LOWER(di.label), r'\bpa?\s*co(?:2|₂)\b')
    )
    AND NOT REGEXP_CONTAINS(LOWER(di.label),
        r'(et\s*co2|end[- ]?tidal|t\s*co2|tco2|total\s*co2|hco3|bicar)')
),
hosp_spec AS (
  SELECT le.specimen_id, LOWER(COALESCE(le.value,'')) AS spec_val
  FROM `{PHYS}.{HOSP}.labevents` le
  JOIN `{PHYS}.{HOSP}.d_labitems` di ON di.itemid = le.itemid
  WHERE le.specimen_id IS NOT NULL
    AND REGEXP_CONTAINS(LOWER(di.label), r'(specimen|sample)')
),
hosp_pco2 AS (
  SELECT
    c.hadm_id, c.charttime,
    CASE
      WHEN REGEXP_CONTAINS(s.spec_val, r'arter') OR REGEXP_CONTAINS(s.spec_val, r'\bart\b') THEN 'arterial'
      WHEN REGEXP_CONTAINS(s.spec_val, r'ven|mixed|central') THEN 'venous'
      WHEN REGEXP_CONTAINS(c.fl, r'arter') THEN 'arterial'
      WHEN REGEXP_CONTAINS(c.fl, r'ven') THEN 'venous'
      WHEN c.fl LIKE '%arterial%' OR REGEXP_CONTAINS(c.lbl, r'\b(abg|art|arterial|a[- ]?line)\b') THEN 'arterial'
      WHEN c.fl LIKE '%ven%'      OR REGEXP_CONTAINS(c.lbl, r'\b(vbg|ven|venous|mixed|central)\b') THEN 'venous'
      ELSE NULL
    END AS site,
    CASE WHEN c.uom_nospace='kpa' THEN c.val*7.50062 ELSE c.val END AS pco2_mmHg
  FROM hosp_cand c
  LEFT JOIN hosp_spec s USING (specimen_id)
  WHERE c.val IS NOT NULL
),
hosp_pco2_std AS (
  SELECT hadm_id, site, charttime, pco2_mmHg
  FROM hosp_pco2
  WHERE site IN ('arterial','venous') AND pco2_mmHg BETWEEN 5 AND 200
),

/* ---- ICU (POC) pCO2 across entire dataset ---- */
icu_raw AS (
  SELECT
    ie.hadm_id,
    ce.stay_id,
    ce.charttime,
    LOWER(di.label) AS lbl,
    LOWER(REPLACE(COALESCE(ce.valueuom,''),' ','')) AS uom_nospace,
    LOWER(COALESCE(ce.value,'')) AS valstr,
    COALESCE(
      CAST(ce.valuenum AS FLOAT64),
      SAFE_CAST(REGEXP_EXTRACT(LOWER(ce.value), r'(-?\d+(?:\.\d+)?)') AS FLOAT64)
    ) AS val
  FROM `{PHYS}.{ICU}.chartevents` ce
  JOIN `{PHYS}.{ICU}.d_items`  di ON di.itemid = ce.itemid
  JOIN `{PHYS}.{ICU}.icustays` ie ON ie.stay_id = ce.stay_id
),
icu_cand AS (
  SELECT
    hadm_id, stay_id, charttime, lbl, uom_nospace, valstr, val,
    CASE
      WHEN (
            REGEXP_CONTAINS(lbl, r'(^|[^a-z])p(?:a)?\s*co\s*(?:2|₂)([^a-z]|$)')
            OR uom_nospace IN ('mmhg','kpa')
            OR REGEXP_CONTAINS(valstr, r'\b(mm\s*hg|kpa)\b')
           )
           AND NOT REGEXP_CONTAINS(lbl,
               r'(et\s*co2|end[- ]?tidal|t\s*co2|tco2|total\s*co2|hco3|bicar|v\s*co2|vco2|co2\s*(prod|elimin|production|elimination))')
      THEN 'pco2'
      ELSE NULL
    END AS analyte,
    CASE
      WHEN REGEXP_CONTAINS(lbl, r'\b(abg|art|arterial|a[- ]?line)\b') THEN 'arterial'
      WHEN REGEXP_CONTAINS(lbl, r'\b(vbg|ven|venous|mixed|central)\b') THEN 'venous'
      ELSE NULL
    END AS site
  FROM icu_raw
  WHERE val IS NOT NULL
    AND (
      REGEXP_CONTAINS(lbl, r'(^|[^a-z])p(?:a)?\s*co\s*(?:2|₂)([^a-z]|$)')
      OR uom_nospace IN ('mmhg','kpa')
      OR REGEXP_CONTAINS(valstr, r'\b(mm\s*hg|kpa)\b')
    )
    AND NOT REGEXP_CONTAINS(lbl,
        r'(et\s*co2|end[- ]?tidal|t\s*co2|tco2|total\s*co2|hco3|bicar|v\s*co2|vco2|co2\s*(prod|elimin|production|elimination))')
),
icu_co2_std AS (
  SELECT
    hadm_id,
    site,
    charttime,
    CASE WHEN uom_nospace='kpa' OR REGEXP_CONTAINS(valstr, r'\bkpa\b') THEN val*7.50062 ELSE val END AS pco2_mmHg
  FROM icu_cand
  WHERE analyte='pco2' AND site IN ('arterial','venous') AND val BETWEEN 5 AND 200
),

/* ---- Combine and threshold per admission ---- */
all_pco2 AS (
  SELECT * FROM hosp_pco2_std
  UNION ALL
  SELECT * FROM icu_co2_std
),
thresh AS (
  SELECT
    hadm_id,
    MAX(IF(site='arterial' AND pco2_mmHg >= 45.0, 1, 0)) AS abg_hypercap_threshold,
    MAX(IF(site='venous'   AND pco2_mmHg >= 50.0, 1, 0)) AS vbg_hypercap_threshold
  FROM all_pco2
  GROUP BY hadm_id
)
SELECT * FROM thresh
"""

co2_thresh = run_sql_bq(sql("co2_thresholds_sql"))
print("Admissions meeting thresholds:", len(co2_thresh))
co2_thresh.head(3)



Admissions meeting thresholds: 52715


,hadm_id,abg_hypercap_threshold,vbg_hypercap_threshold
0,26892251,0,0
1,21943565,0,0
2,27109430,0,0


## 3) Cohort union (ICD ∪ thresholds) and `hadm_list` for downstream queries

**Rationale:** Combine ICD and physiologic routes to maximize sensitivity, then define a stable hadm_id list for downstream joins.


In [7]:
# Outer-join because thresholds can identify hadm_id with no ICD codes and vice versa
cohort_any = cohort_icd.merge(co2_thresh, how="outer", on="hadm_id")

# Fill missing flags with 0 where appropriate
icd_cols = ["ICD10_J9602","ICD10_J9612","ICD10_J9622","ICD10_J9692","ICD10_E662","ICD9_27803","any_hypercap_icd","any_hypercap_icd_hosp","any_hypercap_icd_ed"]
for c in icd_cols:
    if c in cohort_any.columns:
        cohort_any[c] = cohort_any[c].fillna(0).astype(int)

for c in ["abg_hypercap_threshold","vbg_hypercap_threshold"]:
    if c in cohort_any.columns:
        cohort_any[c] = cohort_any[c].fillna(0).astype(int)

# Final enrollment flag
cohort_any["pco2_threshold_any"] = ((cohort_any["abg_hypercap_threshold"]==1) | (cohort_any["vbg_hypercap_threshold"]==1)).astype(int)
cohort_any["enrolled_any"] = ((cohort_any["any_hypercap_icd"]==1) | (cohort_any["pco2_threshold_any"]==1)).astype(int)

print("ICD-only admissions        :", int((cohort_any["any_hypercap_icd"]==1).sum()))
print("Threshold-only admissions  :", int(((cohort_any["pco2_threshold_any"]==1) & (cohort_any["any_hypercap_icd"]==0)).sum()))
print("Both ICD and threshold     :", int(((cohort_any["pco2_threshold_any"]==1) & (cohort_any["any_hypercap_icd"]==1)).sum()))
print("Total enrolled (union)     :", int((cohort_any["enrolled_any"]==1).sum()))

# New hadm list used for the rest of the notebook
hadm_list = cohort_any.loc[cohort_any["enrolled_any"]==1, "hadm_id"].dropna().astype("int64").tolist()
len(hadm_list)



ICD-only admissions        : 4237
Threshold-only admissions  : 35915
Both ICD and threshold     : 3102
Total enrolled (union)     : 40152


40151

## 4) First ABG and First VBG (LAB + POC, standardized to mmHg)

**Rationale:** Extract earliest ABG/VBG measurements to characterize baseline physiology with standardized units.


In [8]:
params = {"hadms": hadm_list}

bg_pairs_sql = rf"""
WITH hadms AS (SELECT hadm_id FROM UNNEST(@hadms) AS hadm_id),

/* ---------------- LAB (HOSP) ---------------- */
hosp_cand AS (
  SELECT
    le.subject_id, le.hadm_id, le.charttime, le.specimen_id,
    CAST(le.valuenum AS FLOAT64) AS val,
    LOWER(COALESCE(le.valueuom,'')) AS uom,
    LOWER(di.label) AS lbl,
    LOWER(COALESCE(di.fluid,'')) AS fl,
    LOWER(COALESCE(di.category,'')) AS cat
  FROM `{PHYS}.{HOSP}.labevents`  le
  JOIN `{PHYS}.{HOSP}.d_labitems` di ON di.itemid = le.itemid
  JOIN hadms h ON h.hadm_id = le.hadm_id
  WHERE le.valuenum IS NOT NULL
    AND (
         LOWER(COALESCE(di.category,'')) LIKE '%blood gas%' OR
         LOWER(di.label) LIKE '%pco2%' OR
         REGEXP_CONTAINS(LOWER(di.label), r'\bph\b') OR
         REGEXP_CONTAINS(LOWER(di.label), r'\bpa?\s*co(?:2|₂)\b')
        )
    AND NOT REGEXP_CONTAINS(LOWER(di.label), r'(et\s*co2|end[- ]?tidal|t\s*co2|tco2|total\s*co2|hco3|bicar)')
),
hosp_spec AS (
  SELECT le.specimen_id, LOWER(COALESCE(le.value,'')) AS spec_val
  FROM `{PHYS}.{HOSP}.labevents` le
  JOIN `{PHYS}.{HOSP}.d_labitems` di ON di.itemid = le.itemid
  WHERE le.specimen_id IS NOT NULL
    AND REGEXP_CONTAINS(LOWER(di.label), r'(specimen|sample)')
),
hosp_class AS (
  SELECT
    c.hadm_id, c.charttime, c.specimen_id, c.val, c.uom, c.lbl, c.fl,
    CASE
      WHEN REGEXP_CONTAINS(c.lbl, r'\b(?:blood\s*)?ph\b') THEN 'ph'
      WHEN (c.lbl LIKE '%pco2%' OR REGEXP_CONTAINS(c.lbl, r'\bpa?\s*co(?:2|₂)\b')) THEN 'pco2'
      ELSE NULL
    END AS analyte,
    CASE
      WHEN REGEXP_CONTAINS(s.spec_val, r'arter') OR REGEXP_CONTAINS(s.spec_val, r'\bart\b') THEN 'arterial'
      WHEN REGEXP_CONTAINS(s.spec_val, r'ven|mixed|central') THEN 'venous'
      WHEN c.fl LIKE '%arterial%' OR REGEXP_CONTAINS(c.lbl, r'\b(abg|art|arterial|a[- ]?line)\b') THEN 'arterial'
      WHEN c.fl LIKE '%ven%'      OR REGEXP_CONTAINS(c.lbl, r'\b(vbg|ven|venous|mixed|central)\b') THEN 'venous'
      ELSE NULL
    END AS site
  FROM hosp_cand c
  LEFT JOIN hosp_spec s USING (specimen_id)
),
hosp_pairs AS (
  SELECT
    hadm_id, specimen_id,
    MIN(charttime) AS sample_time,
    MAX(IF(analyte='ph',   val, NULL)) AS ph,
    MAX(IF(analyte='pco2', val, NULL)) AS pco2_raw,
    (ARRAY_AGG(IF(analyte='pco2', uom, NULL) IGNORE NULLS LIMIT 1))[OFFSET(0)] AS pco2_uom,
    (ARRAY_AGG(IF(analyte='ph',   uom, NULL) IGNORE NULLS LIMIT 1))[OFFSET(0)] AS ph_uom,
    (ARRAY_AGG(site IGNORE NULLS LIMIT 1))[OFFSET(0)] AS site
  FROM hosp_class
  GROUP BY hadm_id, specimen_id
  HAVING (ph IS NOT NULL OR pco2_raw IS NOT NULL) AND site IN ('arterial','venous')
),
hosp_pairs_std AS (
  SELECT
    hadm_id, specimen_id, sample_time, site,
    ph, ph_uom,
    CASE WHEN pco2_uom = 'kpa' THEN pco2_raw * 7.50062 ELSE pco2_raw END AS pco2_mmHg,
    'mmhg' AS pco2_uom_norm
  FROM hosp_pairs
  WHERE (ph IS NULL OR (ph BETWEEN 6.3 AND 7.8))
    AND (pco2_raw IS NULL OR (CASE WHEN pco2_uom='kpa' THEN pco2_raw*7.50062 ELSE pco2_raw END) BETWEEN 5 AND 200)
),
lab_abg AS (
  SELECT hadm_id,
         ph            AS lab_abg_ph,
         ph_uom        AS lab_abg_ph_uom,
         pco2_mmHg     AS lab_abg_paco2,
         'mmhg'        AS lab_abg_paco2_uom,
         sample_time   AS lab_abg_time
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
        FROM hosp_pairs_std WHERE site='arterial') WHERE rn=1
),
lab_vbg AS (
  SELECT hadm_id,
         ph            AS lab_vbg_ph,
         ph_uom        AS lab_vbg_ph_uom,
         pco2_mmHg     AS lab_vbg_paco2,
         'mmhg'        AS lab_vbg_paco2_uom,
         sample_time   AS lab_vbg_time
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
        FROM hosp_pairs_std WHERE site='venous') WHERE rn=1
),

/* ---------------- POC (ICU) ---------------- */
icu_raw AS (
  SELECT
    ie.hadm_id, ce.stay_id, ce.charttime,
    LOWER(di.label) AS lbl,
    LOWER(REPLACE(COALESCE(ce.valueuom,''),' ','')) AS uom_nospace,
    LOWER(COALESCE(ce.value,'')) AS valstr,
    COALESCE(
      CAST(ce.valuenum AS FLOAT64),
      SAFE_CAST(REGEXP_EXTRACT(LOWER(ce.value), r'(-?\d+(?:\.\d+)?)') AS FLOAT64)
    ) AS val
  FROM `{PHYS}.{ICU}.chartevents` ce
  JOIN `{PHYS}.{ICU}.d_items`  di ON di.itemid = ce.itemid
  JOIN `{PHYS}.{ICU}.icustays` ie ON ie.stay_id = ce.stay_id
  JOIN hadms h ON h.hadm_id = ie.hadm_id
),
icu_cand AS (
  SELECT
    hadm_id, stay_id, charttime, lbl, uom_nospace, valstr, val,
    CASE
      WHEN REGEXP_CONTAINS(lbl, r'(^|[^a-z])ph([^a-z]|$)') OR (val BETWEEN 6.3 AND 7.8) THEN 'ph'
      WHEN (
             REGEXP_CONTAINS(lbl, r'(^|[^a-z])p(?:a)?\s*co\s*(?:2|₂)([^a-z]|$)')
             OR uom_nospace IN ('mmhg','kpa')
             OR REGEXP_CONTAINS(valstr, r'\b(mm\s*hg|kpa)\b')
           )
           AND NOT REGEXP_CONTAINS(lbl, r'(et\s*co2|end[- ]?tidal|t\s*co2|tco2|total\s*co2|hco3|bicar|v\s*co2|vco2|co2\s*(prod|elimin|production|elimination))')
      THEN 'pco2'
      ELSE NULL
    END AS analyte,
    CASE
      WHEN REGEXP_CONTAINS(lbl, r'\b(abg|art|arterial|a[- ]?line)\b') THEN 'arterial'
      WHEN REGEXP_CONTAINS(lbl, r'\b(vbg|ven|venous|mixed|central)\b') THEN 'venous'
      ELSE NULL
    END AS site
  FROM icu_raw
  WHERE val IS NOT NULL
    AND (
      REGEXP_CONTAINS(lbl, r'(^|[^a-z])ph([^a-z]|$)') OR
      REGEXP_CONTAINS(lbl, r'(^|[^a-z])p(?:a)?\s*co\s*(?:2|₂)([^a-z]|$)') OR
      uom_nospace IN ('mmhg','kpa') OR
      REGEXP_CONTAINS(valstr, r'\b(mm\s*hg|kpa)\b')
    )
    AND NOT REGEXP_CONTAINS(lbl, r'(et\s*co2|end[- ]?tidal|t\s*co2|tco2|total\s*co2|hco3|bicar|v\s*co2|vco2|co2\s*(prod|elimin|production|elimination))')
),
icu_ph AS (
  SELECT hadm_id, stay_id, charttime, val AS ph, site AS site_ph
  FROM icu_cand WHERE analyte='ph'
),
icu_co2 AS (
  SELECT hadm_id, stay_id, charttime, val AS pco2_raw, uom_nospace, valstr, site AS site_co2
  FROM icu_cand WHERE analyte='pco2'
),
icu_pair_win AS (
  SELECT
    p.hadm_id, p.stay_id,
    COALESCE(p.site_ph, c.site_co2) AS site,
    p.charttime AS ph_time, c.charttime AS co2_time,
    p.ph,
    CASE
      WHEN c.uom_nospace='kpa' OR REGEXP_CONTAINS(c.valstr, r'\bkpa\b') THEN 'kpa'
      WHEN c.uom_nospace='mmhg' OR REGEXP_CONTAINS(c.valstr, r'mm\s*hg') THEN 'mmhg'
      ELSE c.uom_nospace
    END AS pco2_uom_norm_raw,
    c.pco2_raw,
    ABS(TIMESTAMP_DIFF(c.charttime, p.charttime, SECOND)) AS dt_sec
  FROM icu_ph p
  JOIN icu_co2 c
    ON c.hadm_id = p.hadm_id
   AND c.stay_id = p.stay_id
   AND (COALESCE(p.site_ph, c.site_co2) IN ('arterial','venous'))
   AND ABS(TIMESTAMP_DIFF(c.charttime, p.charttime, MINUTE)) <= 10
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY p.hadm_id, p.stay_id, p.charttime
    ORDER BY dt_sec
  ) = 1
),
icu_pairs_std AS (
  SELECT
    hadm_id, stay_id, site,
    LEAST(ph_time, co2_time) AS sample_time,
    ph,
    CAST(NULL AS STRING) AS ph_uom,              -- POC pH is unitless/null
    CASE WHEN pco2_uom_norm_raw='kpa' THEN pco2_raw*7.50062 ELSE pco2_raw END AS pco2_mmHg,
    'mmhg' AS pco2_uom_norm
  FROM icu_pair_win
  WHERE (ph BETWEEN 6.3 AND 7.8 OR ph IS NULL)
    AND (CASE WHEN pco2_uom_norm_raw='kpa' THEN pco2_raw*7.50062 ELSE pco2_raw END) BETWEEN 5 AND 200
),
icu_solo_pco2_std AS (
  SELECT
    hadm_id, stay_id, site_co2 AS site,
    charttime AS sample_time,
    CAST(NULL AS FLOAT64) AS ph,
    CAST(NULL AS STRING)  AS ph_uom,            -- no pH here
    CASE WHEN uom_nospace='kpa' OR REGEXP_CONTAINS(valstr, r'\bkpa\b') THEN pco2_raw*7.50062 ELSE pco2_raw END AS pco2_mmHg,
    'mmhg' AS pco2_uom_norm
  FROM icu_co2
  WHERE site_co2 IN ('arterial','venous')
    AND pco2_raw BETWEEN 5 AND 200
),
icu_all AS (
  SELECT * FROM icu_pairs_std
  UNION ALL
  SELECT * FROM icu_solo_pco2_std
),

poc_abg AS (
  SELECT hadm_id,
         ph            AS poc_abg_ph,
         ph_uom        AS poc_abg_ph_uom,
         pco2_mmHg     AS poc_abg_paco2,
         'mmhg'        AS poc_abg_paco2_uom,
         sample_time   AS poc_abg_time
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
        FROM icu_all WHERE site='arterial') WHERE rn=1
),
poc_vbg AS (
  SELECT hadm_id,
         ph            AS poc_vbg_ph,
         ph_uom        AS poc_vbg_ph_uom,
         pco2_mmHg     AS poc_vbg_paco2,
         'mmhg'        AS poc_vbg_paco2_uom,
         sample_time   AS poc_vbg_time
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
        FROM icu_all WHERE site='venous') WHERE rn=1
)

/* ---------------- Final one row per hadm ---------------- */
SELECT
  h.hadm_id,
  -- LAB-ABG / LAB-VBG
  la.lab_abg_ph, la.lab_abg_ph_uom, la.lab_abg_paco2, la.lab_abg_paco2_uom, la.lab_abg_time,
  lv.lab_vbg_ph, lv.lab_vbg_ph_uom, lv.lab_vbg_paco2, lv.lab_vbg_paco2_uom, lv.lab_vbg_time,
  -- POC-ABG / POC-VBG
  pa.poc_abg_ph, pa.poc_abg_ph_uom, pa.poc_abg_paco2, pa.poc_abg_paco2_uom, pa.poc_abg_time,
  pv.poc_vbg_ph, pv.poc_vbg_ph_uom, pv.poc_vbg_paco2, pv.poc_vbg_paco2_uom, pv.poc_vbg_time,
  -- First ABG across LAB+POC
  (SELECT AS STRUCT src, t, ph, pco2
   FROM (SELECT 'LAB' AS src, la.lab_abg_time AS t, la.lab_abg_ph AS ph, la.lab_abg_paco2 AS pco2
         UNION ALL
         SELECT 'POC', pa.poc_abg_time, pa.poc_abg_ph, pa.poc_abg_paco2)
   WHERE t IS NOT NULL
   ORDER BY t LIMIT 1) AS first_abg,
  -- First VBG across LAB+POC
  (SELECT AS STRUCT src, t, ph, pco2
   FROM (SELECT 'LAB' AS src, lv.lab_vbg_time AS t, lv.lab_vbg_ph AS ph, lv.lab_vbg_paco2 AS pco2
         UNION ALL
         SELECT 'POC', pv.poc_vbg_time, pv.poc_vbg_ph, pv.poc_vbg_paco2)
   WHERE t IS NOT NULL
   ORDER BY t LIMIT 1) AS first_vbg
FROM hadms h
LEFT JOIN lab_abg la USING (hadm_id)
LEFT JOIN lab_vbg lv USING (hadm_id)
LEFT JOIN poc_abg pa USING (hadm_id)
LEFT JOIN poc_vbg pv USING (hadm_id)
"""

bg_pairs = run_sql_bq(bg_pairs_sql, params)

# Flatten STRUCTs for first_abg and first_vbg
for col in ["first_abg","first_vbg"]:
    if col in bg_pairs.columns:
        bg_pairs[f"{col}_src"]  = bg_pairs[col].apply(lambda x: x.get("src") if isinstance(x, dict) else None)
        bg_pairs[f"{col}_time"] = bg_pairs[col].apply(lambda x: x.get("t")   if isinstance(x, dict) else None)
        bg_pairs[f"{col}_ph"]   = bg_pairs[col].apply(lambda x: x.get("ph")  if isinstance(x, dict) else None)
        bg_pairs[f"{col}_pco2"] = bg_pairs[col].apply(lambda x: x.get("pco2")if isinstance(x, dict) else None)
        bg_pairs = bg_pairs.drop(columns=[col])

bg_pairs.head(3)


,hadm_id,lab_abg_ph,lab_abg_ph_uom,lab_abg_paco2,lab_abg_paco2_uom,lab_abg_time,lab_vbg_ph,lab_vbg_ph_uom,lab_vbg_paco2,lab_vbg_paco2_uom,...,poc_vbg_paco2_uom,poc_vbg_time,first_abg_src,first_abg_time,first_abg_ph,first_abg_pco2,first_vbg_src,first_vbg_time,first_vbg_ph,first_vbg_pco2
0,20000094,NaN,None,NaN,None,NaT,7.41,units,47.0,mmhg,...,mmhg,2150-03-02 22:05:00,None,NaT,NaN,NaN,LAB,2150-03-02 15:53:00,7.41,47.0
1,20000694,NaN,None,NaN,None,NaT,7.29,units,43.0,mmhg,...,None,NaT,None,NaT,NaN,NaN,LAB,2144-09-01 05:15:00,7.29,43.0
2,20001305,NaN,None,NaN,None,NaT,7.29,units,77.0,mmhg,...,mmhg,2178-03-25 04:28:00,None,NaT,NaN,NaN,LAB,2178-03-25 04:28:00,7.29,77.0


## 5) Demographics & outcomes

**Rationale:** Build baseline covariates used for descriptive statistics and potential confounding adjustment.


In [9]:

SQL["demo_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x)
SELECT
  a.hadm_id,
  a.subject_id,
  a.admittime,
  a.dischtime,
  a.deathtime,
  a.admission_type,
  a.admission_location,
  a.discharge_location,
  a.insurance,
  -- LOS (days)
  TIMESTAMP_DIFF(a.dischtime, a.admittime, HOUR) / 24.0 AS hosp_los_days,
  -- in-hospital death
  IF(a.deathtime IS NOT NULL, 1, 0) AS death_in_hosp,
  -- demographics
  p.gender,
  SAFE_CAST(ROUND(p.anchor_age + (EXTRACT(YEAR FROM a.admittime) - p.anchor_year), 1) AS FLOAT64) AS age_at_admit,
  -- 30-day all-cause mortality from admission
  IF(p.dod IS NOT NULL AND DATE_DIFF(DATE(p.dod), DATE(a.admittime), DAY) BETWEEN 0 AND 30, 1, 0) AS death_30d
FROM `{PHYS}.{HOSP}.admissions` a
JOIN hadms h USING (hadm_id)
JOIN `{PHYS}.{HOSP}.patients` p USING (subject_id)
"""
demo = run_sql_bq(sql("demo_sql"), {"hadms": hadm_list})
print("Demo rows:", len(demo))
demo.head(3)



Demo rows: 40151


,hadm_id,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,hosp_los_days,death_in_hosp,gender,age_at_admit,death_30d
0,23485217,10584718,2165-02-12 15:41:00,2165-03-06 08:20:00,2165-03-06 08:20:00,EW EMER.,TRANSFER FROM SKILLED NURSING FACILITY,DIED,Medicare,21.708333,1,M,78.0,1
1,22661627,10032409,2130-01-12 18:42:00,2130-01-21 14:32:00,NaT,EW EMER.,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,8.833333,0,F,77.0,0
2,22447711,13352386,2188-03-25 01:17:00,2188-03-26 16:00:00,NaT,EW EMER.,EMERGENCY ROOM,HOME,Private,1.625000,0,F,64.0,0


In [10]:
# ==== Drop-in: safe merge utilities (one cell, run once) ====
import pandas as pd
from typing import Iterable, Optional, Literal

def _ensure_Int64(s: pd.Series) -> pd.Series:
    """Coerce to pandas nullable Int64 (preserves NA)."""
    return pd.to_numeric(s, errors="coerce").astype("Int64")

def strip_subject_cols(fr: pd.DataFrame) -> pd.DataFrame:
    """Remove any subject_id-like columns from a frame (e.g., 'subject_id', 'Subject_ID')."""
    return fr.drop(columns=[c for c in fr.columns if c.lower().startswith("subject_id")],
                   errors="ignore")

def safe_merge_on_hadm(
    left: pd.DataFrame,
    right: pd.DataFrame,
    *,
    right_name: str,
    take: Optional[Iterable[str]] = None,
    order_by: Optional[Iterable[str]] = None,
    check_subject: Literal[False, "warn", "raise"] = False,
) -> pd.DataFrame:
    """
    Left-merge 'right' into 'left' on hadm_id, returning a copy of left with right's columns.
    - Dedupes right on hadm_id (optionally using order_by to pick the first row).
    - Optionally restricts right columns via `take`.
    - Optionally audits subject_id agreement before dropping subject_id from right.
    - Always strips subject_id-like columns from the right to prevent *_x/_y suffixes.
    - Raises if any *_x/_y suffixes still appear (indicates overlapping names besides hadm_id).
    """
    if "hadm_id" not in left.columns:
        raise KeyError(f"left frame lacks hadm_id before merging {right_name}")
    if "hadm_id" not in right.columns:
        raise KeyError(f"{right_name} lacks hadm_id")

    L = left.copy()
    R = right.copy()

    # Standardize dtypes of keys
    L["hadm_id"] = _ensure_Int64(L["hadm_id"])
    R["hadm_id"] = _ensure_Int64(R["hadm_id"])
    if "subject_id" in L.columns:
        L["subject_id"] = _ensure_Int64(L["subject_id"])
    if "subject_id" in R.columns:
        R["subject_id"] = _ensure_Int64(R["subject_id"])

    # Dedupe RIGHT by hadm_id (optionally order_by first)
    if order_by:
        R = (R.sort_values(list(order_by))
               .drop_duplicates(subset=["hadm_id"], keep="first"))
    else:
        R = R.drop_duplicates(subset=["hadm_id"], keep="first")

    # Optional subject_id consistency audit (before stripping)
    if check_subject and ("subject_id" in L.columns) and ("subject_id" in R.columns):
        # Join only on hadm_id where both sides have subject_id
        tmp = (L[["hadm_id", "subject_id"]]
                 .merge(R[["hadm_id", "subject_id"]],
                        on="hadm_id", how="inner", suffixes=("_L","_R")))
        mism = (tmp["subject_id_L"].notna() & tmp["subject_id_R"].notna() &
                (tmp["subject_id_L"] != tmp["subject_id_R"]))
        n_mism = int(mism.sum())
        if n_mism > 0:
            sample_ids = tmp.loc[mism, "hadm_id"].head(10).tolist()
            msg = (f"[{right_name}] subject_id mismatch on {n_mism} hadm_id(s). "
                   f"Examples: {sample_ids}")
            if check_subject == "raise":
                raise ValueError(msg)
            else:
                print("WARNING:", msg)

    # Limit right columns (avoid accidental overlaps)
    if take is not None:
        keep = ["hadm_id"] + [c for c in take if c != "hadm_id"]
        R = R[[c for c in keep if c in R.columns]]

    # Always strip subject_id-like columns from right to prevent *_x/_y
    R = strip_subject_cols(R)

    # Final merge
    out = L.merge(R, on="hadm_id", how="left", suffixes=("", ""))

    # Guard: no suffixes should be present
    bad = [c for c in out.columns if c.endswith("_x") or c.endswith("_y")]
    if bad:
        raise RuntimeError(
            f"Merge with {right_name} produced suffixed columns {bad}. "
            "You likely have overlapping column names other than hadm_id."
        )
    return out

print("Safe merge helpers loaded.")


Safe merge helpers loaded.


## 6) NIH/OMB race & ethnicity (ED + Hospital)

**Rationale:** Harmonize race/ethnicity across sources using NIH/OMB categories for consistent reporting.


In [11]:
race_eth_sql = rf"""
WITH hadms AS (
  SELECT x AS hadm_id
  FROM UNNEST(@hadms) AS x
),

-- Hospital admission "race" text
hosp AS (
  SELECT a.hadm_id, LOWER(TRIM(a.race)) AS race_hosp_raw
  FROM `{PHYS}.{HOSP}.admissions` a
  JOIN hadms hm USING (hadm_id)
),

-- Earliest ED stay leading to the admission; take its "race" text if present
ed_first AS (
  SELECT
    e.hadm_id,
    (ARRAY_AGG(STRUCT(e.intime AS intime, LOWER(TRIM(e.race)) AS race_ed_raw)
               ORDER BY e.intime ASC LIMIT 1))[OFFSET(0)] AS pick
  FROM `{PHYS}.{ED}.edstays` e
  JOIN hadms hm USING (hadm_id)
  GROUP BY e.hadm_id
),
ed AS (
  SELECT hadm_id, pick.race_ed_raw
  FROM ed_first
),

-- Combine ED + Hospital for maximum coverage
comb AS (
  SELECT
    hm.hadm_id,
    ho.race_hosp_raw,
    ed.race_ed_raw,
    TRIM(REGEXP_REPLACE(CONCAT(COALESCE(ho.race_hosp_raw,''), ' ', COALESCE(ed.race_ed_raw,'')), r'\s+', ' ')) AS race_text_any
  FROM hadms hm
  LEFT JOIN hosp ho USING (hadm_id)
  LEFT JOIN ed   ed USING (hadm_id)
),

-- Tokenization to OMB families + Hispanic ethnicity
tok AS (
  SELECT
    hadm_id, race_hosp_raw, race_ed_raw, race_text_any,

    -- Ethnicity (Hispanic)
    REGEXP_CONTAINS(race_text_any, r'\b(hispanic|latinx|latino|latina)\b') AS is_hisp,

    -- Race families (use boundaries to reduce false positives)
    REGEXP_CONTAINS(race_text_any, r'american\s+indian|\balaska\b') AS is_aian,
    REGEXP_CONTAINS(race_text_any, r'\basian\b') AS is_asian,
    REGEXP_CONTAINS(race_text_any, r'\b(black|african\s+american)\b') AS is_black,
    REGEXP_CONTAINS(race_text_any, r'hawaiian|pacific\s+islander') AS is_nhopi,
    REGEXP_CONTAINS(race_text_any, r'\bwhite\b|caucasian') AS is_white,

    -- Unknown/other indicators
    REGEXP_CONTAINS(race_text_any, r'unknown|other|declined|unable|not\s+reported|missing|null') AS is_unknown_any,

    -- Multi-race hints
    REGEXP_CONTAINS(race_text_any, r'(two|2)\s+or\s+more|multi|biracial|multiracial') AS is_multi_hint
  FROM comb
),

-- Decide ethnicity per NIH
ethn AS (
  SELECT
    hadm_id, race_hosp_raw, race_ed_raw, race_text_any,
    CASE
      WHEN is_hisp THEN 'Hispanic or Latino'
      WHEN (race_text_any IS NULL OR race_text_any = '' OR is_unknown_any) THEN 'Unknown or Not Reported'
      ELSE 'Not Hispanic or Latino'
    END AS nih_ethnicity,
    (CAST(is_aian AS INT64) + CAST(is_asian AS INT64) + CAST(is_black AS INT64)
     + CAST(is_nhopi AS INT64) + CAST(is_white AS INT64)) AS race_hits,
    is_aian, is_asian, is_black, is_nhopi, is_white, is_multi_hint, is_unknown_any
  FROM tok
),

-- Decide race per NIH/OMB (1997)
race_assign AS (
  SELECT
    hadm_id, race_hosp_raw, race_ed_raw, race_text_any, nih_ethnicity,
    CASE
      WHEN race_hits >= 2 OR is_multi_hint THEN 'More than one race'
      WHEN is_aian THEN 'American Indian or Alaska Native'
      WHEN is_asian THEN 'Asian'
      WHEN is_black THEN 'Black or African American'
      WHEN is_nhopi THEN 'Native Hawaiian or Other Pacific Islander'
      WHEN is_white THEN 'White'
      WHEN is_unknown_any OR race_text_any IS NULL OR race_text_any = '' THEN 'Unknown or Not Reported'
      ELSE 'Unknown or Not Reported'
    END AS nih_race
  FROM ethn
)

SELECT hadm_id, race_hosp_raw, race_ed_raw, nih_race, nih_ethnicity
FROM race_assign
"""
race_eth = run_sql_bq(race_eth_sql, {"hadms": hadm_list})
print("Race/Eth rows:", len(race_eth))
race_eth.head(3)



Race/Eth rows: 40151


,hadm_id,race_hosp_raw,race_ed_raw,nih_race,nih_ethnicity
0,20000094,white,None,White,Not Hispanic or Latino
1,20000694,white,white,White,Not Hispanic or Latino
2,20001305,white,white,White,Not Hispanic or Latino


## 7) ED triage (linked to hadm) and first ED vitals

**Rationale:** Capture ED presentation features (vitals and chief complaint) for symptom and severity analyses.


In [12]:
# ED triage
SQL["ed_triage_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
edmap AS (
  SELECT stay_id, hadm_id, intime
  FROM `{PHYS}.{ED}.edstays`
  WHERE hadm_id IN (SELECT hadm_id FROM hadms)
),
tri AS (
  SELECT stay_id, temperature, heartrate, resprate, o2sat, sbp, dbp, pain, acuity, chiefcomplaint
  FROM `{PHYS}.{ED}.triage`
),
tri_by_stay AS (
  SELECT m.hadm_id, m.intime, t.*
  FROM edmap m
  JOIN tri t USING (stay_id)
),
tri_by_hadm AS (
  SELECT
    hadm_id,
    (ARRAY_AGG(STRUCT(intime, temperature, heartrate, resprate, o2sat, sbp, dbp, pain, acuity, chiefcomplaint)
               ORDER BY intime LIMIT 1))[OFFSET(0)] AS pick
  FROM tri_by_stay
  GROUP BY hadm_id
)
SELECT
  hadm_id,
  pick.temperature    AS ed_triage_temp,
  pick.heartrate      AS ed_triage_hr,
  pick.resprate       AS ed_triage_rr,
  pick.o2sat          AS ed_triage_o2sat,
  pick.sbp            AS ed_triage_sbp,
  pick.dbp            AS ed_triage_dbp,
  pick.pain           AS ed_triage_pain,
  pick.acuity         AS ed_triage_acuity,
  pick.chiefcomplaint AS ed_triage_cc
FROM tri_by_hadm
"""
ed_triage = run_sql_bq(sql("ed_triage_sql"), {"hadms": hadm_list})
print("ED triage rows:", len(ed_triage))

# First ED vitals
SQL["ed_first_vitals_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
edmap AS (
  SELECT stay_id, hadm_id
  FROM `{PHYS}.{ED}.edstays`
  WHERE hadm_id IN (SELECT hadm_id FROM hadms)
),
vs AS (
  SELECT stay_id, charttime, temperature, heartrate, resprate, o2sat, sbp, dbp, rhythm, pain
  FROM `{PHYS}.{ED}.vitalsign`
),
first_vs AS (
  SELECT
    m.hadm_id,
    (ARRAY_AGG(STRUCT(v.charttime, v.temperature, v.heartrate, v.resprate, v.o2sat, v.sbp, v.dbp, v.rhythm, v.pain)
               ORDER BY v.charttime LIMIT 1))[OFFSET(0)] AS pick
  FROM edmap m JOIN vs v USING (stay_id)
  GROUP BY m.hadm_id
)
SELECT
  hadm_id,
  pick.charttime   AS ed_first_vitals_time,
  pick.temperature AS ed_first_temp,
  pick.heartrate   AS ed_first_hr,
  pick.resprate    AS ed_first_rr,
  pick.o2sat       AS ed_first_o2sat,
  pick.sbp         AS ed_first_sbp,
  pick.dbp         AS ed_first_dbp,
  pick.rhythm      AS ed_first_rhythm,
  pick.pain        AS ed_first_pain
FROM first_vs
"""
ed_first = run_sql_bq(sql("ed_first_vitals_sql"), {"hadms": hadm_list})
print("ED first vitals rows:", len(ed_first))



ED triage rows: 17424
ED first vitals rows: 16746


## 8) ICU meta (first ICU stay, LOS days)

**Rationale:** Summarize ICU exposure and length of stay to contextualize disease severity.


In [13]:
SQL["icu_meta_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
first_icu AS (
  SELECT
    hadm_id,
    (ARRAY_AGG(STRUCT(stay_id, intime, outtime) ORDER BY intime LIMIT 1))[OFFSET(0)] AS s
  FROM `{PHYS}.{ICU}.icustays`
  WHERE hadm_id IN (SELECT hadm_id FROM hadms)
  GROUP BY hadm_id
)
SELECT
  hadm_id,
  s.stay_id AS first_icu_stay_id,
  s.intime  AS icu_intime,
  s.outtime AS icu_outtime,
  TIMESTAMP_DIFF(s.outtime, s.intime, HOUR)/24.0 AS icu_los_days
FROM first_icu
"""
icu_meta = run_sql_bq(sql("icu_meta_sql"), {"hadms": hadm_list})
print("ICU meta rows:", len(icu_meta))



ICU meta rows: 26059


## 9) Ventilation flags (ICD procedures)

**Rationale:** Identify IMV/NIV exposure as clinically relevant respiratory support indicators.


In [14]:
# Cheaper ventilation flags: yes/no for IMV and NIV (ICD-9/ICD-10), plus any_vent_flag
SQL["vent_sql"] = f"""
WITH hadms AS (
  SELECT x AS hadm_id
  FROM UNNEST(@hadms) AS x
),

-- Restrict to admissions of interest early to minimize CPU
proc AS (
  SELECT hadm_id, icd_version, icd_code
  FROM `{PHYS}.{HOSP}.procedures_icd`
  JOIN hadms USING (hadm_id)
),

-- ICU Invasive Mechanical Ventilation (IMV)
-- ICD-10-PCS (usually stored without a dot)
imv10 AS (
  SELECT DISTINCT hadm_id
  FROM proc
  WHERE icd_version = 10
    AND icd_code IN ('5A1935Z','5A1945Z','5A1955Z','0BH17EZ','0BH18EZ')
),

-- ICD-9-CM procedures (stored sometimes with dot, sometimes without)
imv9 AS (
  SELECT DISTINCT hadm_id
  FROM proc
  WHERE icd_version = 9
    AND (
          icd_code IN ('96.70','96.71','96.72','96.04')  -- dotted forms
          OR REPLACE(icd_code, '.', '') IN ('9670','9671','9672','9604') -- dotless match
        )
),

-- Noninvasive Ventilation (NIV)
-- ICD-10-PCS (no dot)
niv10 AS (
  SELECT DISTINCT hadm_id
  FROM proc
  WHERE icd_version = 10
    AND icd_code IN ('5A09357','5A09457','5A09557')
),

-- ICD-9-CM (with/without dot)
niv9 AS (
  SELECT DISTINCT hadm_id
  FROM proc
  WHERE icd_version = 9
    AND (
          icd_code IN ('93.90','93.91','93.99')            -- dotted
          OR REPLACE(icd_code, '.', '') IN ('9390','9391','9399')  -- dotless
        )
)

SELECT
  h.hadm_id,
  IF(i10.hadm_id IS NOT NULL OR i9.hadm_id IS NOT NULL, 1, 0) AS imv_flag,
  IF(n10.hadm_id IS NOT NULL OR n9.hadm_id IS NOT NULL, 1, 0) AS niv_flag,
  -- any_vent = either IMV or NIV
  IF(
    (i10.hadm_id IS NOT NULL OR i9.hadm_id IS NOT NULL)
    OR (n10.hadm_id IS NOT NULL OR n9.hadm_id IS NOT NULL),
    1, 0
  ) AS any_vent_flag
FROM hadms h
LEFT JOIN imv10 i10 ON h.hadm_id = i10.hadm_id
LEFT JOIN imv9  i9  ON h.hadm_id = i9.hadm_id
LEFT JOIN niv10 n10 ON h.hadm_id = n10.hadm_id
LEFT JOIN niv9  n9  ON h.hadm_id = n9.hadm_id
"""

vent = run_sql_bq(sql("vent_sql"), {"hadms": hadm_list})
print("Vent rows:", len(vent))


Vent rows: 40151


and from charts

In [15]:
SQL["vent_chart_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),

stays AS (
  SELECT hadm_id, stay_id
  FROM `{PHYS}.{ICU}.icustays`
  WHERE hadm_id IN (SELECT hadm_id FROM hadms)
),

cand AS (
  SELECT
    s.hadm_id,
    ce.stay_id,
    ce.charttime,
    LOWER(di.label) AS lbl,
    LOWER(COALESCE(ce.value,'')) AS valstr
  FROM `{PHYS}.{ICU}.chartevents` ce
  JOIN `{PHYS}.{ICU}.d_items`    di ON di.itemid = ce.itemid
  JOIN stays s ON s.stay_id = ce.stay_id
  WHERE
    (
      REGEXP_CONTAINS(LOWER(di.label), r'(vent|ventilator|mode|bipap|bi[- ]?pap|cpap|nippv|niv|mask|ett|endotracheal)')
      OR REGEXP_CONTAINS(LOWER(ce.value), r'(bipap|bi[- ]?pap|cpap|nippv|niv|endotracheal|ett|ac/|simv|prvc|aprv|pcv|vcv|assist\s*control)')
    )
),

flags AS (
  SELECT
    hadm_id,
    MIN(IF(
          REGEXP_CONTAINS(lbl,   r'(non[- ]?invasive|niv|nippv|bipap|bi[- ]?pap|cpap)')
          OR REGEXP_CONTAINS(valstr,r'(non[- ]?invasive|niv|nippv|bipap|bi[- ]?pap|cpap)'),
          charttime, NULL)) AS first_niv_time,
    MIN(IF(
          REGEXP_CONTAINS(lbl,   r'(invasive ventilation|endotracheal|ett|mech(|anical)? vent|ventilator mode|ac/|simv|prvc|aprv|pcv|vcv|assist\s*control)')
          OR REGEXP_CONTAINS(valstr,r'(invasive ventilation|endotracheal|ett|ac/|simv|prvc|aprv|pcv|vcv|assist\s*control)'),
          charttime, NULL)) AS first_imv_time,
    MAX( CASE
            WHEN REGEXP_CONTAINS(lbl,   r'(non[- ]?invasive|niv|nippv|bipap|bi[- ]?pap|cpap)')
              OR REGEXP_CONTAINS(valstr,r'(non[- ]?invasive|niv|nippv|bipap|bi[- ]?pap|cpap)')
            THEN 1 ELSE 0
        END ) AS niv_chart_flag,
    MAX( CASE
            WHEN REGEXP_CONTAINS(lbl,   r'(invasive ventilation|endotracheal|ett|mech(|anical)? vent|ventilator mode|ac/|simv|prvc|aprv|pcv|vcv|assist\s*control)')
              OR REGEXP_CONTAINS(valstr,r'(invasive ventilation|endotracheal|ett|ac/|simv|prvc|aprv|pcv|vcv|assist\s*control)')
            THEN 1 ELSE 0
        END ) AS imv_chart_flag
  FROM cand
  GROUP BY hadm_id
)

SELECT hadm_id, niv_chart_flag, imv_chart_flag, first_niv_time, first_imv_time
FROM flags
"""
vent_chart = run_sql_bq(sql("vent_chart_sql"), {"hadms": hadm_list})



In [16]:
# If your existing ICD-only result is called `vent`, rename for clarity:
vent_proc = vent.copy()

# Outer merge so we keep hadm_ids that appear in only one source
vent_combined = vent_proc.merge(vent_chart, on="hadm_id", how="outer")

# Fill missing with 0 before taking maxima
for c in ["imv_flag","niv_flag","any_vent_flag","imv_chart_flag","niv_chart_flag"]:
    if c in vent_combined.columns:
        vent_combined[c] = vent_combined[c].fillna(0).astype("Int64")

# Final "any-source" flags
vent_combined["imv_flag"]       = vent_combined[["imv_flag","imv_chart_flag"]].max(axis=1).astype("Int64")
vent_combined["niv_flag"]       = vent_combined[["niv_flag","niv_chart_flag"]].max(axis=1).astype("Int64")
vent_combined["any_vent_flag"]  = vent_combined[["imv_flag","niv_flag"]].max(axis=1).astype("Int64")

vent_combined = vent_combined[["hadm_id","imv_flag","niv_flag","any_vent_flag","first_imv_time","first_niv_time"]]
print("After combining ICD + chart signals:",
      "\nIMV=1:", int((vent_combined["imv_flag"]==1).sum()),
      "\nNIV=1:", int((vent_combined["niv_flag"]==1).sum()))



After combining ICD + chart signals: 
IMV=1: 23655 
NIV=1: 10540


## 10) Assemble final DataFrame

**Rationale:** Merge all derived features into a single analytic table keyed by hadm_id.


In [17]:
# Canonical base (carries authoritative subject_id)
df = demo.copy()

# Cohort flags / thresholds / labs / etc.
df = safe_merge_on_hadm(df, cohort_any, right_name="cohort_any", check_subject="warn")
df = safe_merge_on_hadm(df, bg_pairs,   right_name="bg_pairs")
df = safe_merge_on_hadm(df, race_eth,   right_name="race_eth")
df = safe_merge_on_hadm(df, ed_triage,  right_name="ed_triage")
df = safe_merge_on_hadm(df, ed_first,   right_name="ed_first")
df = safe_merge_on_hadm(df, icu_meta,   right_name="icu_meta")
df = safe_merge_on_hadm(df, vent_combined,       right_name="vent_combined")

# Anchor to first ED presentation (per admission)
if "ed_intime_first" in globals():
    df = safe_merge_on_hadm(df, ed_intime_first, right_name="ed_intime_first")

# Derived timing: first NIV/IMV relative to ED presentation
if "ed_intime_first" in df.columns and "first_imv_time" in df.columns:
    df["dt_first_imv_hours"] = (df["first_imv_time"] - df["ed_intime_first"]).dt.total_seconds() / 3600.0
if "ed_intime_first" in df.columns and "first_niv_time" in df.columns:
    df["dt_first_niv_hours"] = (df["first_niv_time"] - df["ed_intime_first"]).dt.total_seconds() / 3600.0

# ABG/VBG before IMV (hadm-level)
if {"first_abg_time", "first_imv_time"}.issubset(df.columns):
    df["abg_before_imv"] = (
        df["first_abg_time"].notna() & df["first_imv_time"].notna() &
        (df["first_abg_time"] < df["first_imv_time"])
    ).astype("Int64")
if {"first_vbg_time", "first_imv_time"}.issubset(df.columns):
    df["vbg_before_imv"] = (
        df["first_vbg_time"].notna() & df["first_imv_time"].notna() &
        (df["first_vbg_time"] < df["first_imv_time"])
    ).astype("Int64")

print("Final df rows:", len(df), "cols:", len(df.columns))

# Safety checks
assert "subject_id" in df.columns, "subject_id missing from final df"
assert not any(c.endswith("_x") or c.endswith("_y") for c in df.columns), "Found suffixed columns"
print("Final df rows:", len(df), "cols:", len(df.columns))
df.head(3)



Final df rows: 40151 cols: 89
Final df rows: 40151 cols: 89


,hadm_id,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,hosp_los_days,...,icu_intime,icu_outtime,icu_los_days,imv_flag,niv_flag,any_vent_flag,first_imv_time,first_niv_time,abg_before_imv,vbg_before_imv
0,23485217,10584718,2165-02-12 15:41:00,2165-03-06 08:20:00,2165-03-06 08:20:00,EW EMER.,TRANSFER FROM SKILLED NURSING FACILITY,DIED,Medicare,21.708333,...,2165-02-27 21:41:10,2165-03-06 11:09:58,6.583333,1,0,1,2165-02-27 22:00:00,NaT,0,1
1,22661627,10032409,2130-01-12 18:42:00,2130-01-21 14:32:00,NaT,EW EMER.,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,8.833333,...,NaT,NaT,NaN,0,0,0,NaT,NaT,0,0
2,22447711,13352386,2188-03-25 01:17:00,2188-03-26 16:00:00,NaT,EW EMER.,EMERGENCY ROOM,HOME,Private,1.625000,...,NaT,NaT,NaN,0,0,0,NaT,NaT,0,0


In [18]:
# --- Cohort flow counts (ED / ICU / blood gas / hypercapnia / CC) ---

# 1) Dataset-level ED counts
SQL["ed_counts_sql"] = f"""
SELECT
  COUNT(*) AS total_ed_encounters,
  COUNTIF(hadm_id IS NOT NULL) AS ed_encounters_with_hadm
FROM `{PHYS}.{ED}.edstays`
"""

SQL["ed_to_icu_sql"] = f"""
SELECT
  COUNT(DISTINCT e.hadm_id) AS ed_to_icu_hadm,
  COUNT(DISTINCT e.stay_id) AS ed_to_icu_edstays
FROM `{PHYS}.{ED}.edstays` e
JOIN `{PHYS}.{ICU}.icustays` i USING (hadm_id)
WHERE e.hadm_id IS NOT NULL
"""

try:
    ed_counts = run_sql_bq(sql("ed_counts_sql"))
    ed_to_icu = run_sql_bq(sql("ed_to_icu_sql"))
except Exception as e:
    print("Warning: ED/ICU counts query failed:", e)
    ed_counts = None
    ed_to_icu = None

# 2) Cohort-level counts (admission-level)
cohort_union = int((cohort_any["enrolled_any"] == 1).sum()) if "cohort_any" in globals() and "enrolled_any" in cohort_any.columns else len(hadm_list)
cohort_df_n = len(df)

# Any blood gas present (ABG/VBG, LAB/POC)
co2_cols = [c for c in [
    "lab_abg_paco2", "lab_vbg_paco2", "poc_abg_paco2", "poc_vbg_paco2"
] if c in df.columns]
any_bg = int(df[co2_cols].notna().any(axis=1).sum()) if co2_cols else None

# Hypercapnia thresholds and ICD
icd_count = int((df["any_hypercap_icd"] == 1).sum()) if "any_hypercap_icd" in df.columns else None
threshold_count = int((df["pco2_threshold_any"] == 1).sum()) if "pco2_threshold_any" in df.columns else None

# ED chief complaint missing / present (within cohort)
if "ed_triage_cc" in df.columns:
    mask_cc_present = df["ed_triage_cc"].notna() & (df["ed_triage_cc"].astype(str).str.strip() != "")
    cc_present = int(mask_cc_present.sum())
    cc_missing = int((~mask_cc_present).sum())
else:
    cc_present = None
    cc_missing = None

# ED→ICU within cohort (admissions with ED triage data and ICU stay)
if "first_icu_stay_id" in df.columns and "ed_triage_cc" in df.columns:
    cohort_ed_to_icu = int((df["first_icu_stay_id"].notna() & mask_cc_present).sum())
else:
    cohort_ed_to_icu = None

rows = []
if ed_counts is not None:
    rows.append({"step": "Total ED encounters (edstays)", "count": int(ed_counts.loc[0, "total_ed_encounters"]), "scope": "All ED dataset"})
    rows.append({"step": "ED encounters with hadm_id", "count": int(ed_counts.loc[0, "ed_encounters_with_hadm"]), "scope": "All ED dataset"})
if ed_to_icu is not None:
    rows.append({"step": "ED→ICU admissions (distinct hadm_id)", "count": int(ed_to_icu.loc[0, "ed_to_icu_hadm"]), "scope": "All ED+ICU"})
    rows.append({"step": "ED→ICU ED-stays (distinct stay_id)", "count": int(ed_to_icu.loc[0, "ed_to_icu_edstays"]), "scope": "All ED+ICU"})

rows.append({"step": "Cohort admissions (union ICD ∪ thresholds)", "count": cohort_union, "scope": "Cohort"})
rows.append({"step": "Cohort admissions after merges (df rows)", "count": cohort_df_n, "scope": "Cohort"})
if any_bg is not None:
    rows.append({"step": "Cohort with any ABG/VBG (LAB or POC)", "count": any_bg, "scope": "Cohort"})
if threshold_count is not None:
    rows.append({"step": "Cohort meeting hypercapnia thresholds", "count": threshold_count, "scope": "Cohort"})
if icd_count is not None:
    rows.append({"step": "Cohort meeting ICD code criteria", "count": icd_count, "scope": "Cohort"})
if cc_present is not None:
    rows.append({"step": "Cohort with ED chief complaint present", "count": cc_present, "scope": "Cohort"})
if cc_missing is not None:
    rows.append({"step": "Cohort excluded for missing ED chief complaint", "count": cc_missing, "scope": "Cohort"})
if cohort_ed_to_icu is not None:
    rows.append({"step": "Cohort ED→ICU (ED CC present + ICU stay)", "count": cohort_ed_to_icu, "scope": "Cohort"})

flow_counts = pd.DataFrame(rows)
flow_counts



,step,count,scope
0,Total ED encounters (edstays),425087,All ED dataset
1,ED encounters with hadm_id,203016,All ED dataset
2,ED→ICU admissions (distinct hadm_id),31862,All ED+ICU
3,ED→ICU ED-stays (distinct stay_id),31916,All ED+ICU
4,Cohort admissions (union ICD ∪ thresholds),40152,Cohort
5,Cohort admissions after merges (df rows),40151,Cohort
6,Cohort with any ABG/VBG (LAB or POC),37887,Cohort
7,Cohort meeting hypercapnia thresholds,39016,Cohort
8,Cohort meeting ICD code criteria,4237,Cohort
9,Cohort with ED chief complaint present,17424,Cohort


In [19]:
# --- Ascertainment overlap counts (ABG/VBG/ICD) ---

required = ["abg_hypercap_threshold", "vbg_hypercap_threshold", "any_hypercap_icd"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Missing required columns for overlap counts: {missing}")

abg = pd.to_numeric(df["abg_hypercap_threshold"], errors="coerce").fillna(0).astype(int)
vbg = pd.to_numeric(df["vbg_hypercap_threshold"], errors="coerce").fillna(0).astype(int)

gas_any = (
    pd.to_numeric(df.get("pco2_threshold_any", None), errors="coerce")
    if "pco2_threshold_any" in df.columns else (abg | vbg)
)
if hasattr(gas_any, "fillna"):
    gas_any = gas_any.fillna(0).astype(int)
else:
    gas_any = gas_any.astype(int)

icd = pd.to_numeric(df["any_hypercap_icd"], errors="coerce").fillna(0).astype(int)

total_n = len(df)
ngas = int((gas_any == 1).sum())

abg_vbg_overlap = pd.DataFrame([
    {"group": "ABG-only", "count": int(((abg==1) & (vbg==0)).sum())},
    {"group": "VBG-only", "count": int(((vbg==1) & (abg==0)).sum())},
    {"group": "ABG+VBG", "count": int(((abg==1) & (vbg==1)).sum())},
])
if ngas > 0:
    abg_vbg_overlap["pct_of_gas"] = (abg_vbg_overlap["count"] / ngas * 100).round(1)
else:
    abg_vbg_overlap["pct_of_gas"] = 0.0
abg_vbg_overlap["pct_of_cohort"] = (abg_vbg_overlap["count"] / max(total_n,1) * 100).round(1)

icd_gas_overlap = pd.DataFrame([
    {"group": "ICD+Gas", "count": int(((icd==1) & (gas_any==1)).sum())},
    {"group": "ICD-only", "count": int(((icd==1) & (gas_any==0)).sum())},
    {"group": "Gas-only", "count": int(((icd==0) & (gas_any==1)).sum())},
    {"group": "Neither", "count": int(((icd==0) & (gas_any==0)).sum())},
])
icd_gas_overlap["pct_of_cohort"] = (icd_gas_overlap["count"] / max(total_n,1) * 100).round(1)

print("ABG/VBG overlap (among gas-positive):")
print(abg_vbg_overlap.to_string(index=False))
print("ICD vs Gas overlap (cohort-level):")
print(icd_gas_overlap.to_string(index=False))



ABG/VBG overlap (among gas-positive):
   group  count  pct_of_gas  pct_of_cohort
ABG-only      0         0.0            0.0
VBG-only  39016       100.0           97.2
 ABG+VBG      0         0.0            0.0
ICD vs Gas overlap (cohort-level):
   group  count  pct_of_cohort
 ICD+Gas   3102            7.7
ICD-only   1135            2.8
Gas-only  35914           89.4
 Neither      0            0.0


In [20]:
# --- Missingness summary (chief complaint, race/ethnicity, ED triage/vitals) ---

import numpy as np

summary_rows = []

# Chief complaint missingness (ED triage CC)
if "ed_triage_cc" in df.columns:
    cc_present = df["ed_triage_cc"].notna() & (df["ed_triage_cc"].astype(str).str.strip() != "")
    summary_rows.append({
        "variable": "ed_triage_cc_present",
        "missing_n": int((~cc_present).sum()),
        "missing_pct": float((~cc_present).mean())
    })

# Race/Ethnicity missingness (NIH categories + raw sources)
unknown_tokens = {
    "unknown or not reported",
    "unknown",
    "not reported",
    "missing",
    "declined",
    "unable"
}

def _missing_rate(series):
    if series is None:
        return None, None
    s = series.astype(str).str.strip()
    is_missing = series.isna() | (s == "") | s.str.lower().isin(unknown_tokens)
    return int(is_missing.sum()), float(is_missing.mean())

for col in ["nih_race", "nih_ethnicity", "race_hosp_raw", "race_ed_raw"]:
    if col in df.columns:
        m_n, m_p = _missing_rate(df[col])
        summary_rows.append({
            "variable": col,
            "missing_n": m_n,
            "missing_pct": m_p
        })

missing_summary = pd.DataFrame(summary_rows)
print("Missingness summary (key variables):")
missing_summary

# ED triage + first ED vitals missingness
triage_cols = [c for c in df.columns if c.startswith("ed_triage_")]
first_cols  = [c for c in df.columns if c.startswith("ed_first_")]

vital_cols = triage_cols + first_cols
if vital_cols:
    miss_tbl = (
        pd.DataFrame({"variable": vital_cols})
        .assign(
            missing_n=lambda d: [int(df[c].isna().sum()) for c in d["variable"]],
            missing_pct=lambda d: [float(df[c].isna().mean()) for c in d["variable"]]
        )
        .sort_values("missing_pct", ascending=False)
    )
    print("Missingness summary (ED triage + first ED vitals):")
    miss_tbl
else:
    miss_tbl = pd.DataFrame(columns=["variable", "missing_n", "missing_pct"])



Missingness summary (key variables):
Missingness summary (ED triage + first ED vitals):


## 11) Sanity checks

**Rationale:** Run QC checks to validate units, flags, and basic data integrity before export.


In [21]:

# ICU LOS negative?
if {"icu_los_days","first_icu_stay_id"}.issubset(df.columns):
    neg_los = int((df["icu_los_days"] < 0).fillna(False).sum())
    print("Negative ICU LOS rows:", neg_los)

# Vent flags consistency
vent_cols = {"imv_flag","niv_flag","any_vent_flag"}
if vent_cols.issubset(df.columns):
    any_calc = ((df["imv_flag"]==1) | (df["niv_flag"]==1)).fillna(False).astype(int)
    any_flag = pd.to_numeric(df["any_vent_flag"], errors="coerce").fillna(0).astype(int)
    mism = int((any_calc != any_flag).sum())
    print("any_vent_flag mismatches vs (imv|niv):", mism)

# UOMs: expect mmhg only
uom_cols = [c for c in df.columns if c.endswith("_paco2_uom")]
for c in uom_cols:
    vals = sorted(pd.Series(df[c]).dropna().astype(str).str.lower().str.strip().unique().tolist())
    print(c, vals)

# ABG/VBG coverage QC
def qc_pair(df, ph_col, co2_col, label, ph_lo=6.3, ph_hi=7.8, co2_lo=5, co2_hi=200):
    ph  = pd.to_numeric(df.get(ph_col), errors="coerce")
    co2 = pd.to_numeric(df.get(co2_col), errors="coerce")
    return {
        "pair": label,
        "present_any":  int(((ph.notna()) | (co2.notna())).sum()),
        "present_both": int(((ph.notna()) & (co2.notna())).sum()),
        "only_ph":      int(((ph.notna()) & (~co2.notna())).sum()),
        "only_pco2":    int(((co2.notna()) & (~ph.notna())).sum()),
        "ph_oob":       int((((ph  < ph_lo)  | (ph  > ph_hi))  & ph.notna()).sum()),
        "pco2_oob":     int((((co2 < co2_lo) | (co2 > co2_hi)) & co2.notna()).sum()),
    }

qc = pd.DataFrame([
    qc_pair(df, "lab_abg_ph","lab_abg_paco2","LAB ABG"),
    qc_pair(df, "lab_vbg_ph","lab_vbg_paco2","LAB VBG"),
    qc_pair(df, "poc_abg_ph","poc_abg_paco2","POC ABG"),
    qc_pair(df, "poc_vbg_ph","poc_vbg_paco2","POC VBG"),
])
qc


Negative ICU LOS rows: 0
any_vent_flag mismatches vs (imv|niv): 0
lab_abg_paco2_uom ['mmhg']
lab_vbg_paco2_uom ['mmhg']
poc_abg_paco2_uom ['mmhg']
poc_vbg_paco2_uom ['mmhg']


,pair,present_any,present_both,only_ph,only_pco2,ph_oob,pco2_oob
0,LAB ABG,17368,17168,195,5,0,0
1,LAB VBG,36980,33600,3372,8,0,0
2,POC ABG,15411,7046,0,8365,0,0
3,POC VBG,22413,16476,0,5937,0,0


## 12) Save to Excel

**Rationale:** Persist cohort outputs for downstream annotation and NLP analyses.


In [22]:
from datetime import datetime
out_path = DATA_DIR / f"mimic_hypercap_EXT_bq_abg_vbg_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as xw:
    df.to_excel(xw, sheet_name="cohort", index=False)
    try:
        qc.to_excel(xw, sheet_name="qc_abg_vbg", index=False)
    except Exception:
        pass
out_path



PosixPath('/Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/mimic_hypercap_EXT_bq_abg_vbg_20260204_210911.xlsx')

## Create Annotation Dataset

**Rationale:** Create ED chief-complaint subsets and a reproducible sample for manual annotation.


In [23]:
# ---- Extra exports: (1) ED chief-complaint only; (2) random sample of 160 patients ----
from datetime import datetime

# Where to write
out_dir = DATA_DIR
out_dir.mkdir(parents=True, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 1) Filter to rows with a non-empty ED chief complaint
if "ed_triage_cc" not in df.columns:
    raise KeyError(
        "Column 'ed_triage_cc' not found in df. "
        "Ensure the ED triage merge cell ran earlier."
    )

mask_cc = df["ed_triage_cc"].notna() & (df["ed_triage_cc"].astype(str).str.strip() != "")
df_cc = df.loc[mask_cc].copy()

print(f"ED-CC present rows: {len(df_cc)} of {len(df)} "
      f"({(len(df_cc) / max(len(df),1)):.1%} of cohort).")

# Save ED-CC-only cohort
out_path_cc = out_dir / f"mimic_hypercap_EXT_EDcc_only_bq_abg_vbg_{timestamp}.xlsx"
with pd.ExcelWriter(out_path_cc, engine="openpyxl") as xw:
    df_cc.to_excel(xw, sheet_name="cohort_cc_only", index=False)
    try:
        qc.to_excel(xw, sheet_name="qc_abg_vbg", index=False)
    except Exception:
        pass
print("Saved:", out_path_cc)

# 2) Random sample of n = 160 patients (distinct subject_id), one row per patient
if "subject_id" not in df_cc.columns:
    raise KeyError("Column 'subject_id' missing; cannot sample by patient.")

# Make a one-row-per-patient frame by earliest admission
if "admittime" in df_cc.columns:
    df_cc_one = (
        df_cc.sort_values(["subject_id", "admittime"])
             .groupby("subject_id", as_index=False)
             .head(1)
    )
else:
    # Fallback if admittime not present: choose the smallest hadm_id per patient
    df_cc_one = (
        df_cc.sort_values(["subject_id", "hadm_id"])
             .groupby("subject_id", as_index=False)
             .head(1)
    )

N = 160
n_avail = len(df_cc_one)
n_take = min(N, n_avail)
if n_avail < N:
    print(f"Warning: only {n_avail} unique patients with ED chief complaint; sampling all of them.")

RANDOM_SEED = 42
df_cc_sample = df_cc_one.sample(n=n_take, random_state=RANDOM_SEED)

# Save the sample
out_path_cc_sample = out_dir / f"mimic_hypercap_EXT_EDcc_sample{n_take}_bq_abg_vbg_{timestamp}.xlsx"
with pd.ExcelWriter(out_path_cc_sample, engine="openpyxl") as xw:
    df_cc_sample.to_excel(xw, sheet_name="cohort_cc_sample", index=False)
    try:
        qc.to_excel(xw, sheet_name="qc_abg_vbg", index=False)
    except Exception:
        pass
print("Saved:", out_path_cc_sample)



ED-CC present rows: 17424 of 40151 (43.4% of cohort).
Saved: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/mimic_hypercap_EXT_EDcc_only_bq_abg_vbg_20260204_210948.xlsx
Saved: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/mimic_hypercap_EXT_EDcc_sample160_bq_abg_vbg_20260204_210948.xlsx


# ED-stay cohort expansion (timing, severity, comorbidity, outcomes)

**Rationale:** Build a one-row-per-ED-stay analytic extract with time-anchored gas phenotypes, key comorbidities, and outcomes.


## Phase 0 — Inventory & missing-field registry

**Rationale:** Detect which fields already exist and only add missing fields to avoid redundant extraction or join explosions.


In [24]:
from pathlib import Path
import json
import pandas as pd

# Identify current cohort dataframe (admission-level)
if 'df' not in globals():
    raise NameError("Expected admission-level df to exist before inventory step.")

ED_KEY = "ed_stay_id"  # target key for ED-level cohort

print("Current admission-level df:", df.shape)
print("Current columns count:", len(df.columns))
if ED_KEY in df.columns:
    print("ED stay unique count:", int(df[ED_KEY].nunique()))
else:
    print("ED stay unique count: ED_KEY not in columns")

# Persist columns snapshot
cols_out = WORK_DIR / "current_columns.json"
cols_out.write_text(json.dumps(sorted(df.columns), indent=2))
print("Wrote:", cols_out)



# Persist ED-stay columns snapshot (if ed_df exists)
if "ed_df" in globals():
    ed_cols_out = WORK_DIR / "ed_columns.json"
    ed_cols_out.write_text(json.dumps(sorted(ed_df.columns), indent=2))
    print("Wrote:", ed_cols_out)


Current admission-level df: (40151, 89)
Current columns count: 89
ED stay unique count: ED_KEY not in columns
Wrote: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/current_columns.json


In [25]:
# Target field registry
TARGET_RAW_FIELDS = {
    "ed_edstays": [
        "ed_stay_id","subject_id","hadm_id","ed_intime","ed_outtime","ed_intime_first",
        "arrival_transport","disposition","ed_gender","ed_race",
    ],
    "ed_triage": [
        "ed_triage_temp","ed_triage_hr","ed_triage_rr","ed_triage_o2sat",
        "ed_triage_sbp","ed_triage_dbp","ed_triage_pain","ed_triage_acuity","ed_triage_cc",
    ],
    "ed_vitals_first": [
        "ed_first_vitals_time","ed_first_temp","ed_first_hr","ed_first_rr",
        "ed_first_o2sat","ed_first_sbp","ed_first_dbp","ed_first_rhythm","ed_first_pain",
    ],
    "admissions": [
        "admittime","dischtime","deathtime","hospital_expire_flag",
        "admission_type","admission_location","discharge_location",
        "insurance","language","marital_status","hosp_race",
    ],
    "icu": [
        "icu_stay_id","icu_intime_first","icu_outtime_last","icu_los_total","n_icu_stays",
        "first_careunit","last_careunit",
    ],
    "labs_gas": [
        "first_gas_time","first_pco2","first_ph","first_hco3","first_lactate",
        "max_pco2_0_6h","min_ph_0_6h","max_pco2_0_24h","min_ph_0_24h",
        "flag_abg_hypercapnia","flag_vbg_hypercapnia","flag_any_gas_hypercapnia",
        "gas_source_unknown_rate",
        "dt_first_imv_hours","dt_first_niv_hours",
    ],
    "omr": [
        "bmi_closest_pre_ed","height_closest_pre_ed","weight_closest_pre_ed",
    ],
    "dx_flags": [
        "flag_copd","flag_osa_ohs","flag_chf","flag_neuromuscular",
        "flag_opioid_substance","flag_pneumonia",
    ],
    "timing": [
        "dt_first_qualifying_gas_hours","presenting_hypercapnia","late_hypercapnia",
        "dt_first_imv_hours","dt_first_niv_hours","abg_before_imv","vbg_before_imv",
        "ph_band","hco3_band","lactate_band",
    ],
}

TARGET_DERIVED_FIELDS = [
    "hospital_los_hours","in_hospital_death",
]

TARGET_FIELDS = sorted({c for v in TARGET_RAW_FIELDS.values() for c in v} | set(TARGET_DERIVED_FIELDS))

# Grouped missing report
missing_by_group = {}
for group, cols in TARGET_RAW_FIELDS.items():
    missing_by_group[group] = [c for c in cols if c not in df.columns]

missing_derived = [c for c in TARGET_DERIVED_FIELDS if c not in df.columns]
missing_by_group["derived"] = missing_derived

missing_flat = [c for cols in missing_by_group.values() for c in cols]
print("Missing fields total:", len(missing_flat))
for group, cols in missing_by_group.items():
    if cols:
        print(f"- {group}: {cols}")





Missing fields total: 53
- ed_edstays: ['ed_stay_id', 'ed_intime', 'ed_outtime', 'ed_intime_first', 'arrival_transport', 'disposition', 'ed_gender', 'ed_race']
- admissions: ['hospital_expire_flag', 'language', 'marital_status', 'hosp_race']
- icu: ['icu_stay_id', 'icu_intime_first', 'icu_outtime_last', 'icu_los_total', 'n_icu_stays', 'first_careunit', 'last_careunit']
- labs_gas: ['first_gas_time', 'first_pco2', 'first_ph', 'first_hco3', 'first_lactate', 'max_pco2_0_6h', 'min_ph_0_6h', 'max_pco2_0_24h', 'min_ph_0_24h', 'flag_abg_hypercapnia', 'flag_vbg_hypercapnia', 'flag_any_gas_hypercapnia', 'gas_source_unknown_rate', 'dt_first_imv_hours', 'dt_first_niv_hours']
- omr: ['bmi_closest_pre_ed', 'height_closest_pre_ed', 'weight_closest_pre_ed']
- dx_flags: ['flag_copd', 'flag_osa_ohs', 'flag_chf', 'flag_neuromuscular', 'flag_opioid_substance', 'flag_pneumonia']
- timing: ['dt_first_qualifying_gas_hours', 'presenting_hypercapnia', 'late_hypercapnia', 'dt_first_imv_hours', 'dt_first_niv_ho

## Phase 1 — ED encounter spine and ED enrichment (one row per ED stay)

**Rationale:** Create a dedicated ED-stay-level cohort with ED-specific attributes to avoid mixing admission- and ED-level grains.


In [26]:
# ED stay spine (rename stay_id to ed_stay_id)

SQL["ed_spine_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x)
SELECT
  s.stay_id AS ed_stay_id,
  s.subject_id,
  s.hadm_id,
  s.intime AS ed_intime,
  s.outtime AS ed_outtime,
  s.arrival_transport,
  s.disposition,
  s.gender AS ed_gender,
  s.race   AS ed_race
FROM `{PHYS}.{ED}.edstays` s
JOIN hadms h ON h.hadm_id = s.hadm_id
"""

ed_spine = run_sql_bq(sql("ed_spine_sql"), {"hadms": hadm_list})
print("ED spine rows:", len(ed_spine), "unique ed_stay_id:", ed_spine["ed_stay_id"].nunique())

# ensure uniqueness
if ed_spine["ed_stay_id"].nunique() != len(ed_spine):
    raise ValueError("ed_stay_id not unique in ED spine")

# First ED presentation time per admission
ed_intime_first = (
    ed_spine.groupby("hadm_id", as_index=False)["ed_intime"]
    .min()
    .rename(columns={"ed_intime": "ed_intime_first"})
)

# Start ED-level df
ed_df = ed_spine.copy()
ed_df = ed_df.merge(ed_intime_first, on="hadm_id", how="left")



ED spine rows: 17449 unique ed_stay_id: 17449


In [27]:
# ED triage and first ED vitals (reuse existing logic if present; otherwise join)

def _needs_cols(df, cols):
    return (df is None) or any(c not in df.columns for c in cols)

# Use existing ed_triage / ed_first if already in memory from earlier cells
try:
    _ = ed_triage
except NameError:
    ed_triage = None

try:
    _ = ed_first
except NameError:
    ed_first = None

# Force re-query if required keys are missing
if _needs_cols(locals().get('ed_triage', None), ['ed_stay_id', 'hadm_id']):
    ed_triage = None
if _needs_cols(locals().get('ed_first', None), ['ed_stay_id']):
    ed_first = None

# If missing, re-run ED triage / first vitals queries with ed_stay_id output
if ed_triage is None:
    ed_triage_sql = f"""
    WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x)
    SELECT
      s.stay_id AS ed_stay_id,
      s.hadm_id,
      s.intime,
      t.temperature    AS ed_triage_temp,
      t.heartrate      AS ed_triage_hr,
      t.resprate       AS ed_triage_rr,
      t.o2sat          AS ed_triage_o2sat,
      t.sbp            AS ed_triage_sbp,
      t.dbp            AS ed_triage_dbp,
      t.pain           AS ed_triage_pain,
      t.acuity         AS ed_triage_acuity,
      t.chiefcomplaint AS ed_triage_cc
    FROM `{PHYS}.{ED}.edstays` s
    JOIN hadms h ON h.hadm_id = s.hadm_id
    LEFT JOIN `{PHYS}.{ED}.triage` t
      ON t.stay_id = s.stay_id
    """
    ed_triage = run_sql_bq(sql("ed_triage_sql"), {'hadms': hadm_list})
    print('ED triage rows:', len(ed_triage))

if ed_first is None:
    ed_first_vitals_sql = f"""
    WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
    edmap AS (
      SELECT stay_id, hadm_id
      FROM `{PHYS}.{ED}.edstays`
      WHERE hadm_id IN (SELECT hadm_id FROM hadms)
    ),
    vs AS (
      SELECT stay_id, charttime, temperature, heartrate, resprate, o2sat, sbp, dbp, rhythm, pain
      FROM `{PHYS}.{ED}.vitalsign`
    ),
    first_vs AS (
      SELECT
        v.stay_id,
        (ARRAY_AGG(STRUCT(v.charttime, v.temperature, v.heartrate, v.resprate, v.o2sat, v.sbp, v.dbp, v.rhythm, v.pain)
                   ORDER BY v.charttime LIMIT 1))[OFFSET(0)] AS pick
      FROM vs v
      JOIN edmap m USING (stay_id)
      GROUP BY v.stay_id
    )
    SELECT
      f.stay_id AS ed_stay_id,
      pick.charttime AS ed_first_vitals_time,
      pick.temperature AS ed_first_temp,
      pick.heartrate AS ed_first_hr,
      pick.resprate AS ed_first_rr,
      pick.o2sat AS ed_first_o2sat,
      pick.sbp AS ed_first_sbp,
      pick.dbp AS ed_first_dbp,
      pick.rhythm AS ed_first_rhythm,
      pick.pain AS ed_first_pain
    FROM first_vs f
    """
    ed_first = run_sql_bq(sql("ed_first_vitals_sql"), {'hadms': hadm_list})
    print('ED first vitals rows:', len(ed_first))

# Debug columns before merge
print('ed_triage cols:', list(ed_triage.columns))
print('ed_first cols:', list(ed_first.columns))
print('ed_df cols:', list(ed_df.columns))

if 'ed_stay_id' not in ed_df.columns:
    raise KeyError('ed_df missing ed_stay_id; ensure ED spine cell ran.')

# Merge ED triage + vitals onto ed_df with available keys
merge_keys_triage = [k for k in ["ed_stay_id", "hadm_id"] if k in ed_df.columns and k in ed_triage.columns]
if not merge_keys_triage:
    raise KeyError("No common keys between ed_df and ed_triage")
ed_df = ed_df.merge(ed_triage, on=merge_keys_triage, how="left")
merge_keys_first = [k for k in ["ed_stay_id", "hadm_id"] if k in ed_df.columns and k in ed_first.columns]
if not merge_keys_first:
    raise KeyError("No common keys between ed_df and ed_first")
ed_df = ed_df.merge(ed_first, on=merge_keys_first, how="left")






ED triage rows: 17424
ED first vitals rows: 16746
ed_triage cols: ['hadm_id', 'ed_triage_temp', 'ed_triage_hr', 'ed_triage_rr', 'ed_triage_o2sat', 'ed_triage_sbp', 'ed_triage_dbp', 'ed_triage_pain', 'ed_triage_acuity', 'ed_triage_cc']
ed_first cols: ['hadm_id', 'ed_first_vitals_time', 'ed_first_temp', 'ed_first_hr', 'ed_first_rr', 'ed_first_o2sat', 'ed_first_sbp', 'ed_first_dbp', 'ed_first_rhythm', 'ed_first_pain']
ed_df cols: ['ed_stay_id', 'subject_id', 'hadm_id', 'ed_intime', 'ed_outtime', 'arrival_transport', 'disposition', 'ed_gender', 'ed_race', 'ed_intime_first']


## Phase 2 — Hospital admission context and outcomes

**Rationale:** Add admission-level outcomes and demographics for downstream stratification and analysis.


In [28]:
# Admissions fields
SQL["admit_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x)
SELECT
  a.hadm_id,
  a.admittime,
  a.dischtime,
  a.deathtime,
  a.hospital_expire_flag,
  a.admission_type,
  a.admission_location,
  a.discharge_location,
  a.insurance,
  a.language,
  a.marital_status,
  a.race AS hosp_race
FROM `{PHYS}.{HOSP}.admissions` a
JOIN hadms h USING (hadm_id)
"""

admit = run_sql_bq(sql("admit_sql"), {"hadms": hadm_list})
print("Admissions rows:", len(admit))

ed_df = ed_df.merge(admit, on="hadm_id", how="left")

# Merge ventilation flags/times (hadm-level)
if "vent_combined" in globals():
    ed_df = ed_df.merge(vent_combined, on="hadm_id", how="left")

# Derived outcomes
ed_df["hospital_los_hours"] = (ed_df["dischtime"] - ed_df["admittime"]).dt.total_seconds() / 3600.0
ed_df["in_hospital_death"] = ((ed_df["hospital_expire_flag"] == 1) | ed_df["deathtime"].notna()).astype("int64")

# Concordance check
discord = (
    ((ed_df["hospital_expire_flag"] == 1) & ed_df["deathtime"].isna()) |
    ((ed_df["hospital_expire_flag"] == 0) & ed_df["deathtime"].notna())
)
print("Admissions discordance (expire_flag vs deathtime):", int(discord.sum()))



Admissions rows: 40151
Admissions discordance (expire_flag vs deathtime): 3


## Phase 3 — ICU timing and LOS

**Rationale:** Capture ICU exposure, timing, and total LOS for severity stratification.


In [29]:
# ICU stays (aggregate per hadm)
SQL["icu_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x)
SELECT
  i.hadm_id,
  i.stay_id AS icu_stay_id,
  i.intime,
  i.outtime,
  i.los,
  i.first_careunit,
  i.last_careunit
FROM `{PHYS}.{ICU}.icustays` i
JOIN hadms h USING (hadm_id)
"""

icu = run_sql_bq(sql("icu_sql"), {"hadms": hadm_list})
print("ICU stay rows:", len(icu))

if len(icu) > 0:
    icu_agg = (
        icu.sort_values(["hadm_id", "intime"]).groupby("hadm_id", as_index=False)
        .agg(
            icu_intime_first=("intime", "min"),
            icu_outtime_last=("outtime", "max"),
            icu_los_total=("los", "sum"),
            n_icu_stays=("icu_stay_id", "nunique"),
            first_careunit=("first_careunit", "first"),
            last_careunit=("last_careunit", "last"),
        )
    )
    ed_df = ed_df.merge(icu_agg, on="hadm_id", how="left")
else:
    print("No ICU stays found for cohort.")



ICU stay rows: 31393


## Phase 4 — ED longitudinal vitals (0–6h)

**Rationale:** Summarize early ED vitals for severity phenotyping.


In [30]:
# ED vitals long + aggregates (0-6h)

SQL["ed_vitals_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
edmap AS (
  SELECT stay_id AS ed_stay_id, hadm_id, intime AS ed_intime
  FROM `{PHYS}.{ED}.edstays`
  WHERE hadm_id IN (SELECT hadm_id FROM hadms)
)
SELECT
  v.stay_id AS ed_stay_id,
  m.hadm_id,
  m.ed_intime,
  v.charttime,
  v.temperature,
  v.heartrate,
  v.resprate,
  v.o2sat,
  v.sbp,
  v.dbp,
  v.rhythm,
  v.pain
FROM `{PHYS}.{ED}.vitalsign` v
JOIN edmap m ON m.ed_stay_id = v.stay_id
"""

ed_vitals_long = run_sql_bq(sql("ed_vitals_sql"), {"hadms": hadm_list})
print("ED vitals long rows:", len(ed_vitals_long))

# Window filter: 0–6h from ED intime
ed_vitals_long["dt_hours"] = (ed_vitals_long["charttime"] - ed_vitals_long["ed_intime"]).dt.total_seconds() / 3600.0
in_6h = ed_vitals_long["dt_hours"].between(0, 6, inclusive="both")

agg = (
    ed_vitals_long.loc[in_6h]
    .groupby("ed_stay_id", as_index=False)
    .agg(
        max_heartrate_0_6h=("heartrate", "max"),
        max_resprate_0_6h=("resprate", "max"),
        min_o2sat_0_6h=("o2sat", "min"),
        min_sbp_0_6h=("sbp", "min"),
        n_vitals_0_6h=("charttime", "count"),
    )
)

ed_df = ed_df.merge(agg, on="ed_stay_id", how="left")

# Range warnings (do not drop)
range_checks = {
    "heartrate": (0, 300),
    "resprate": (0, 80),
    "o2sat": (0, 100),
    "sbp": (0, 300),
}
for col, (lo, hi) in range_checks.items():
    bad = ed_vitals_long[col].notna() & (~ed_vitals_long[col].between(lo, hi))
    if bad.any():
        print(f"Warning: {col} out of range count:", int(bad.sum()))



ED vitals long rows: 120310


## Phase 5 — Robust lab discovery + gas panels

**Rationale:** Capture blood gas and key chemistry labs with label-robust item discovery and unit normalization.


In [31]:
import re
import json

# Discover itemids from d_labitems
SQL["labitems_sql"] = f"""
SELECT itemid, label, fluid, category
FROM `{PHYS}.{HOSP}.d_labitems`
"""

labitems = run_sql_bq(sql("labitems_sql"))

patterns = {
    "gas_pco2": re.compile(r"\bp\s*co2\b|pco2|pco₂", re.I),
    "gas_ph": re.compile(r"\bph\b", re.I),
    "gas_hco3": re.compile(r"hco3|bicarbonate", re.I),
    "gas_lactate": re.compile(r"lactate", re.I),
    "gas_specimen": re.compile(r"specimen|source|type", re.I),
    "chem_creatinine": re.compile(r"creatinine", re.I),
    "chem_sodium": re.compile(r"\bsodium\b", re.I),
    "chem_chloride": re.compile(r"\bchloride\b", re.I),
    "chem_total_co2": re.compile(r"carbon dioxide|total co2|\bco2\b", re.I),
    "cbc_hemoglobin": re.compile(r"hemoglobin", re.I),
}

# category filters
cat_gas = re.compile(r"blood\s*gas|blood gas|arterial|venous", re.I)
cat_chem = re.compile(r"chemistry|chem|blood", re.I)
cat_cbc = re.compile(r"hematology|cbc", re.I)

matches = {}
for name, pat in patterns.items():
    dfm = labitems.copy()
    dfm = dfm[dfm["label"].str.contains(pat, na=False)]
    if name.startswith("gas_"):
        dfm = dfm[dfm["category"].str.contains(cat_gas, na=False)]
    elif name.startswith("chem_"):
        dfm = dfm[dfm["category"].str.contains(cat_chem, na=False)]
    elif name.startswith("cbc_"):
        dfm = dfm[dfm["category"].str.contains(cat_cbc, na=False)]
    matches[name] = dfm[["itemid","label","category"]]

# Build lab_item_map with counts in cohort
itemids_all = sorted({int(i) for dfm in matches.values() for i in dfm["itemid"].tolist()})

SQL["counts_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x)
SELECT itemid, COUNT(*) AS n
FROM `{PHYS}.{HOSP}.labevents`
WHERE hadm_id IN (SELECT hadm_id FROM hadms)
  AND itemid IN UNNEST(@itemids)
GROUP BY itemid
"""

counts = run_sql_bq(sql("counts_sql"), {"hadms": hadm_list, "itemids": itemids_all}) if itemids_all else pd.DataFrame(columns=["itemid","n"])

lab_item_map = {}
for name, dfm in matches.items():
    tmp = dfm.merge(counts, on="itemid", how="left").fillna({"n":0})
    lab_item_map[name] = {
        "pattern": patterns[name].pattern,
        "items": tmp.sort_values("n", ascending=False).to_dict(orient="records"),
    }

lab_item_map_path = WORK_DIR / "lab_item_map.json"
lab_item_map_path.write_text(json.dumps(lab_item_map, indent=2))
print("Wrote:", lab_item_map_path)




Wrote: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/lab_item_map.json


In [32]:
# Extract labevents within ED windows

# assemble itemid lists
itemid_sets = {k: [int(x["itemid"]) for x in v["items"]] for k, v in lab_item_map.items()}
# flatten
all_itemids = sorted({i for v in itemid_sets.values() for i in v})

SQL["labs_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
eds AS (
  SELECT stay_id AS ed_stay_id, hadm_id, intime AS ed_intime
  FROM `{PHYS}.{ED}.edstays`
  WHERE hadm_id IN (SELECT hadm_id FROM hadms)
)
SELECT
  e.ed_stay_id,
  l.subject_id,
  l.hadm_id,
  l.itemid,
  l.charttime,
  l.specimen_id,
  l.valuenum,
  l.valueuom
FROM `{PHYS}.{HOSP}.labevents` l
JOIN eds e ON e.hadm_id = l.hadm_id
WHERE l.itemid IN UNNEST(@itemids)
  AND l.charttime BETWEEN e.ed_intime AND TIMESTAMP_ADD(e.ed_intime, INTERVAL 24 HOUR)
"""

labs_long = run_sql_bq(sql("labs_sql"), {"hadms": hadm_list, "itemids": all_itemids})
print("Labs long rows:", len(labs_long))

# Unit audit for pCO2
pco2_ids = itemid_sets.get("gas_pco2", [])
unit_audit = (
    labs_long.loc[labs_long["itemid"].isin(pco2_ids)]
    .groupby("valueuom", dropna=False)
    .size().reset_index(name="n")
)
unit_audit_path = WORK_DIR / "lab_unit_audit.csv"
unit_audit.to_csv(unit_audit_path, index=False)
print("Wrote:", unit_audit_path)



Labs long rows: 277691
Wrote: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/lab_unit_audit.csv


In [33]:
# Reconstruct gas panels by specimen_id within ED stay

pco2_ids = set(itemid_sets.get("gas_pco2", []))
ph_ids = set(itemid_sets.get("gas_ph", []))
hco3_ids = set(itemid_sets.get("gas_hco3", []))
lact_ids = set(itemid_sets.get("gas_lactate", []))

labs = labs_long.copy()

# Convert pCO2 kPa to mmHg when needed
is_kpa = labs["valueuom"].astype(str).str.lower().str.contains("kpa", na=False)
mask_pco2 = labs["itemid"].isin(pco2_ids)
if mask_pco2.any() and is_kpa.any():
    labs.loc[mask_pco2 & is_kpa, "valuenum"] = labs.loc[mask_pco2 & is_kpa, "valuenum"] * 7.50062
    labs.loc[mask_pco2 & is_kpa, "valueuom"] = "mmHg"

# panel by specimen_id
panel = (
    labs.groupby(["ed_stay_id","specimen_id"], as_index=False)
    .agg(panel_time=("charttime","min"))
)

# attach analytes

def pick_analyte(df, ids, name):
    tmp = df.loc[df["itemid"].isin(ids), ["ed_stay_id","specimen_id","valuenum"]]
    tmp = tmp.rename(columns={"valuenum": name})
    return tmp.groupby(["ed_stay_id","specimen_id"], as_index=False).first()

if pco2_ids:
    panel = panel.merge(pick_analyte(labs, pco2_ids, "pco2"), on=["ed_stay_id","specimen_id"], how="left")
if ph_ids:
    panel = panel.merge(pick_analyte(labs, ph_ids, "ph"), on=["ed_stay_id","specimen_id"], how="left")
if hco3_ids:
    panel = panel.merge(pick_analyte(labs, hco3_ids, "hco3"), on=["ed_stay_id","specimen_id"], how="left")
if lact_ids:
    panel = panel.merge(pick_analyte(labs, lact_ids, "lactate"), on=["ed_stay_id","specimen_id"], how="left")

# First panel per ED stay
first_panel = (
    panel.sort_values(["ed_stay_id","panel_time"]).groupby("ed_stay_id", as_index=False).first()
)

# 0–6h and 0–24h extrema
panel = panel.merge(ed_df[["ed_stay_id","ed_intime"]], on="ed_stay_id", how="left")
panel["dt_hours"] = (panel["panel_time"] - panel["ed_intime"]).dt.total_seconds() / 3600.0

# Ensure expected panel columns exist even if analyte is absent
for col in ["pco2", "ph", "hco3", "lactate"]:
    if col not in panel.columns:
        panel[col] = pd.NA

p06 = panel.loc[panel["dt_hours"].between(0,6, inclusive="both")]
p24 = panel.loc[panel["dt_hours"].between(0,24, inclusive="both")]

agg06 = p06.groupby("ed_stay_id", as_index=False).agg(max_pco2_0_6h=("pco2","max"), min_ph_0_6h=("ph","min"))
agg24 = p24.groupby("ed_stay_id", as_index=False).agg(max_pco2_0_24h=("pco2","max"), min_ph_0_24h=("ph","min"))

ed_df = ed_df.merge(first_panel[["ed_stay_id","panel_time","pco2","ph","hco3","lactate"]].rename(
    columns={"panel_time":"first_gas_time","pco2":"first_pco2","ph":"first_ph","hco3":"first_hco3","lactate":"first_lactate"}
), on="ed_stay_id", how="left")

ed_df = ed_df.merge(agg06, on="ed_stay_id", how="left")
ed_df = ed_df.merge(agg24, on="ed_stay_id", how="left")




In [34]:
# ABG vs VBG classification (specimen- and label-based inference)

# Use label heuristics from d_labitems if available (arterial/venous hints)
labitems_map = (
    labitems.set_index("itemid")[["label", "fluid"]]
    .fillna("")
    .to_dict("index")
)

def _infer_source_text(text: str) -> str | None:
    if re.search(r"(arterial|abg|a[- ]?line|\bart\b)", text):
        return "arterial"
    if re.search(r"(venous|vbg|central|mixed|\bven\b)", text):
        return "venous"
    return None


def infer_source_item(itemid):
    meta = labitems_map.get(itemid, {})
    text = f"{meta.get('label','')} {meta.get('fluid','')}".lower()
    return _infer_source_text(text)

# Infer source per specimen_id using ANY item label within the specimen
spec_source = None
if "specimen_id" in labs.columns and labs["specimen_id"].notna().any():
    spec_items = labs.loc[labs["specimen_id"].notna(), ["ed_stay_id", "specimen_id", "itemid"]].drop_duplicates()
    spec_items["source_hint"] = spec_items["itemid"].map(infer_source_item)

    def _resolve_source(s):
        s = s.dropna()
        if (s == "arterial").any():
            return "arterial"
        if (s == "venous").any():
            return "venous"
        return "unknown"

    spec_source = (
        spec_items.groupby(["ed_stay_id", "specimen_id"], as_index=False)["source_hint"]
        .apply(_resolve_source)
        .rename(columns={"source_hint": "source"})
    )

if spec_source is not None:
    panel = panel.merge(spec_source, on=["ed_stay_id", "specimen_id"], how="left")

# Fallback: assign source based on pco2 item label
if pco2_ids:
    pco2_itemid = (
        labs.loc[labs["itemid"].isin(pco2_ids), ["ed_stay_id", "specimen_id", "itemid"]]
        .groupby(["ed_stay_id", "specimen_id"], as_index=False)["itemid"].first()
    )
    pco2_itemid["source_pco2"] = pco2_itemid["itemid"].map(infer_source_item)
    panel = panel.merge(pco2_itemid[["ed_stay_id", "specimen_id", "source_pco2"]], on=["ed_stay_id", "specimen_id"], how="left")

    if "source" in panel.columns:
        panel["source"] = panel["source"].where(panel["source"] != "unknown", pd.NA)
        panel["source"] = panel["source"].fillna(panel["source_pco2"])
    else:
        panel["source"] = panel["source_pco2"]
    panel = panel.drop(columns=["source_pco2"])
else:
    if "source" not in panel.columns:
        panel["source"] = "unknown"

panel["source"] = panel["source"].fillna("unknown")

# flags
panel["flag_abg_hypercapnia"] = ((panel["source"]=="arterial") & (panel["pco2"]>=45)).astype(int)
panel["flag_vbg_hypercapnia"] = ((panel["source"]=="venous") & (panel["pco2"]>=50)).astype(int)
panel["flag_any_gas_hypercapnia"] = ((panel["pco2"]>=45)).astype(int)

# per-stay unknown source rate
if len(panel) > 0:
    unk_rate = (
        panel.assign(_unk=(panel["source"].fillna("unknown") == "unknown"))
             .groupby("ed_stay_id", as_index=False)["_unk"].mean()
             .rename(columns={"_unk": "gas_source_unknown_rate"})
    )
else:
    unk_rate = panel[["ed_stay_id"]].drop_duplicates()
    unk_rate["gas_source_unknown_rate"] = 1.0

# collapse to ED stay flags
flags = panel.groupby("ed_stay_id", as_index=False).agg(
    flag_abg_hypercapnia=("flag_abg_hypercapnia","max"),
    flag_vbg_hypercapnia=("flag_vbg_hypercapnia","max"),
    flag_any_gas_hypercapnia=("flag_any_gas_hypercapnia","max"),
)

ed_df = ed_df.merge(flags, on="ed_stay_id", how="left")
ed_df = ed_df.merge(unk_rate, on="ed_stay_id", how="left")
ed_df["gas_source_unknown_rate"] = ed_df["gas_source_unknown_rate"].fillna(1.0)


/var/folders/vf/n84t7b8171lf64smq4ddvw1m0000gn/T/ipykernel_95316/2163351449.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  panel["source"] = panel["source"].fillna(panel["source_pco2"])


## Phase 5C — ICU POC blood gases (chartevents, optional)

**Rationale:** Capture ICU point-of-care gases if central lab labevents miss early measurements.


In [35]:
# Discover ICU POC itemids from d_items

SQL["ditems_sql"] = f"""
SELECT itemid, label, category
FROM `{PHYS}.{ICU}.d_items`
"""

ditems = run_sql_bq(sql("ditems_sql"))

icu_patterns = {
    "pco2": re.compile(r"p\s*co2|pco2|pco₂", re.I),
    "ph": re.compile(r"ph", re.I),
    "hco3": re.compile(r"hco3|bicarbonate", re.I),
    "lactate": re.compile(r"lactate", re.I),
    "specimen": re.compile(r"specimen|source|type", re.I),
}

icu_cat = re.compile(r"blood\s*gas|blood gas|resp|arterial|venous", re.I)

icu_matches = {}
for name, pat in icu_patterns.items():
    dfm = ditems[ditems["label"].str.contains(pat, na=False)]
    dfm = dfm[dfm["category"].str.contains(icu_cat, na=False)]
    icu_matches[name] = dfm[["itemid","label","category"]]

icu_itemids = sorted({int(i) for dfm in icu_matches.values() for i in dfm["itemid"].tolist()})
print("ICU POC candidate itemids:", len(icu_itemids))



ICU POC candidate itemids: 6


In [36]:
# Extract ICU chartevents within ED 0–24h window for cohort ICU stays

if len(icu_itemids) == 0:
    icu_poc_long = pd.DataFrame()
    print("No ICU POC itemids found.")
else:
    # ensure icu stay ids available
    if 'icu' not in globals():
        raise NameError("ICU stays table 'icu' not found; run ICU phase first.")

    icu_stays = icu[["icu_stay_id","hadm_id","intime"]].copy()
    icu_stays = icu_stays.dropna(subset=["icu_stay_id"])

    icu_poc_sql = f"""
    WITH icu_stays AS (
      SELECT stay_id AS icu_stay_id, hadm_id
      FROM `{PHYS}.{ICU}.icustays`
      WHERE stay_id IN UNNEST(@icu_stay_ids)
    ),
    eds AS (
      SELECT stay_id AS ed_stay_id, hadm_id, intime AS ed_intime
      FROM `{PHYS}.{ED}.edstays`
      WHERE hadm_id IN (SELECT hadm_id FROM icu_stays)
    )
    SELECT
      s.icu_stay_id,
      e.ed_stay_id,
      e.hadm_id,
      ce.charttime,
      ce.itemid,
      ce.valuenum,
      ce.valueuom
    FROM `{PHYS}.{ICU}.chartevents` ce
    JOIN icu_stays s ON s.icu_stay_id = ce.stay_id
    JOIN eds e ON e.hadm_id = s.hadm_id
    WHERE ce.itemid IN UNNEST(@itemids)
      AND ce.charttime BETWEEN e.ed_intime AND TIMESTAMP_ADD(e.ed_intime, INTERVAL 24 HOUR)
    """

    icu_poc_long = run_sql_bq(icu_poc_sql, {"icu_stay_ids": icu_stays["icu_stay_id"].astype(int).tolist(), "itemids": icu_itemids})
    print("ICU POC long rows:", len(icu_poc_long))

# Build panels by 5-minute bins per ICU stay
if len(icu_poc_long) > 0:
    icu_poc_long["time_bin"] = icu_poc_long["charttime"].dt.floor("5min")

    def pick_from_ids(df, ids, name):
        tmp = df.loc[df["itemid"].isin(ids), ["icu_stay_id","time_bin","valuenum"]]
        tmp = tmp.rename(columns={"valuenum": name})
        return tmp.groupby(["icu_stay_id","time_bin"], as_index=False).first()

    pco2_ids = set(icu_matches.get("pco2", pd.DataFrame()).get("itemid", []).tolist())
    ph_ids = set(icu_matches.get("ph", pd.DataFrame()).get("itemid", []).tolist())
    hco3_ids = set(icu_matches.get("hco3", pd.DataFrame()).get("itemid", []).tolist())
    lact_ids = set(icu_matches.get("lactate", pd.DataFrame()).get("itemid", []).tolist())

    panel_poc = (
        icu_poc_long.groupby(["icu_stay_id","time_bin"], as_index=False)
        .agg(panel_time=("charttime","min"))
    )
    if pco2_ids:
        panel_poc = panel_poc.merge(pick_from_ids(icu_poc_long, pco2_ids, "pco2"), on=["icu_stay_id","time_bin"], how="left")
    if ph_ids:
        panel_poc = panel_poc.merge(pick_from_ids(icu_poc_long, ph_ids, "ph"), on=["icu_stay_id","time_bin"], how="left")
    if hco3_ids:
        panel_poc = panel_poc.merge(pick_from_ids(icu_poc_long, hco3_ids, "hco3"), on=["icu_stay_id","time_bin"], how="left")
    if lact_ids:
        panel_poc = panel_poc.merge(pick_from_ids(icu_poc_long, lact_ids, "lactate"), on=["icu_stay_id","time_bin"], how="left")

    # map to ED stay via hadm_id
    panel_poc = panel_poc.merge(icu[["icu_stay_id","hadm_id"]], on="icu_stay_id", how="left")
    panel_poc = panel_poc.merge(ed_df[["ed_stay_id","hadm_id","ed_intime"]], on="hadm_id", how="left")
    panel_poc["dt_hours"] = (panel_poc["panel_time"] - panel_poc["ed_intime"]).dt.total_seconds() / 3600.0
else:
    panel_poc = pd.DataFrame()

# Ensure expected panel_poc columns exist even if analyte is absent
for col in ["pco2", "ph", "hco3", "lactate"]:
    if col not in panel_poc.columns:
        panel_poc[col] = pd.NA

poc_flags = pd.DataFrame(columns=["ed_stay_id", "flag_any_gas_hypercapnia_poc"])
if len(panel_poc) > 0:
    p24_poc = panel_poc.loc[panel_poc["dt_hours"].between(0,24, inclusive="both")].copy()
    p24_poc.loc[:, "flag_any_gas_hypercapnia_poc"] = (p24_poc["pco2"] >= 45).astype("Int64")

    poc_flags = p24_poc.groupby("ed_stay_id", as_index=False).agg(
        flag_any_gas_hypercapnia_poc=("flag_any_gas_hypercapnia_poc","max")
    )

# Avoid duplicate column on re-run
if "flag_any_gas_hypercapnia_poc" in ed_df.columns:
    ed_df = ed_df.drop(columns=["flag_any_gas_hypercapnia_poc"])

if len(poc_flags) > 0:
    ed_df = ed_df.merge(poc_flags, on="ed_stay_id", how="left")

# incremental yield
base = ed_df.get("flag_any_gas_hypercapnia", pd.Series(0, index=ed_df.index)).fillna(0).astype(int)
poc = ed_df.get("flag_any_gas_hypercapnia_poc", pd.Series(0, index=ed_df.index)).fillna(0).astype(int)
inc = ((base == 0) & (poc == 1)).sum()
print("ICU POC incremental hypercapnia cases (ED stays):", int(inc))

# optional export
if len(panel_poc) > 0:
    panel_poc.to_parquet(DATA_DIR / "gas_panels_poc.parquet", index=False)


ICU POC long rows: 45400
ICU POC incremental hypercapnia cases (ED stays): 0


## Phase 6 — BMI/anthropometrics (OMR)

**Rationale:** Add BMI/height/weight closest to ED presentation when available.


In [37]:
# OMR BMI/height/weight

try:
    omr_sql = f"""
    SELECT subject_id, chartdate, result_name, result_value
    FROM `{PHYS}.{HOSP}.omr`
    WHERE LOWER(result_name) IN ('bmi','height','weight')
    """
    omr = run_sql_bq(omr_sql)
    print("OMR rows:", len(omr))

    if len(omr) == 0:
        print("OMR query returned 0 rows; BMI/height/weight will remain missing.")
    else:
        omr["result_name"] = omr["result_name"].str.lower()
        omr["result_value_num"] = (
            omr["result_value"].astype(str)
            .str.extract(r"(-?\d+(?:\.\d+)?)", expand=False)
        )
        omr["result_value_num"] = pd.to_numeric(omr["result_value_num"], errors="coerce")

        omr_pivot = (
            omr.pivot_table(
                index=["subject_id","chartdate"],
                columns="result_name",
                values="result_value_num",
                aggfunc="first",
            )
            .reset_index()
        )

        # attach closest pre-ED (within 365 days)
        ed_dates = ed_df[["ed_stay_id","subject_id","ed_intime"]].copy()
        ed_dates["ed_date"] = ed_dates["ed_intime"].dt.date
        omr_pivot["chartdate"] = pd.to_datetime(omr_pivot["chartdate"]).dt.date

        merged = ed_dates.merge(omr_pivot, on="subject_id", how="left")
        merged["days_before"] = (pd.to_datetime(merged["ed_date"]) - pd.to_datetime(merged["chartdate"])).dt.days
        merged = merged.loc[(merged["days_before"] >= 0) & (merged["days_before"] <= 365)]

        closest = (
            merged.sort_values(["ed_stay_id","days_before"]).groupby("ed_stay_id", as_index=False).first()
            .rename(columns={
                "bmi":"bmi_closest_pre_ed",
                "height":"height_closest_pre_ed",
                "weight":"weight_closest_pre_ed",
            })
        )

        ed_df = ed_df.merge(
            closest[["ed_stay_id","bmi_closest_pre_ed","height_closest_pre_ed","weight_closest_pre_ed"]],
            on="ed_stay_id", how="left"
        )
except Exception as e:
    print("OMR not available or failed:", e)



OMR rows: 46494


## Phase 7 — ICD comorbidity flags

**Rationale:** Derive comorbidity indicators from index admission ICD codes for stratified analyses.


In [38]:
# ICD code pulls for comorbidity flags (hospital + ED; combined OR)
# NOTE: Use prefix filters to reduce CPU and avoid regex-heavy scans.

FLAGS = [
    "flag_copd","flag_osa_ohs","flag_chf","flag_neuromuscular",
    "flag_opioid_substance","flag_pneumonia",
]

SQL["icd_flags_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
icd AS (
  SELECT
    hadm_id,
    UPPER(REPLACE(icd_code, \".\", \"\")) AS code_norm
  FROM `{PHYS}.{HOSP}.diagnoses_icd`
  WHERE hadm_id IN (SELECT hadm_id FROM hadms)
    AND (
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J43\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J44\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"G473\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"E662\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"I50\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"G12\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"G70\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"G71\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"F11\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"T40\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"F13\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J12\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J13\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J14\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J15\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J16\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J17\") OR
      STARTS_WITH(UPPER(REPLACE(icd_code, \".\", \"\")), \"J18\")
    )
)
SELECT
  hadm_id,
  MAX(IF(STARTS_WITH(code_norm, \"J43\") OR STARTS_WITH(code_norm, \"J44\"), 1, 0)) AS flag_copd,
  MAX(IF(STARTS_WITH(code_norm, \"G473\") OR STARTS_WITH(code_norm, \"E662\"), 1, 0)) AS flag_osa_ohs,
  MAX(IF(STARTS_WITH(code_norm, \"I50\"), 1, 0)) AS flag_chf,
  MAX(IF(STARTS_WITH(code_norm, \"G12\") OR STARTS_WITH(code_norm, \"G70\") OR STARTS_WITH(code_norm, \"G71\"), 1, 0)) AS flag_neuromuscular,
  MAX(IF(STARTS_WITH(code_norm, \"F11\") OR STARTS_WITH(code_norm, \"T40\") OR STARTS_WITH(code_norm, \"F13\"), 1, 0)) AS flag_opioid_substance,
  MAX(IF(STARTS_WITH(code_norm, \"J12\") OR STARTS_WITH(code_norm, \"J13\") OR STARTS_WITH(code_norm, \"J14\") OR STARTS_WITH(code_norm, \"J15\") OR STARTS_WITH(code_norm, \"J16\") OR STARTS_WITH(code_norm, \"J17\") OR STARTS_WITH(code_norm, \"J18\"), 1, 0)) AS flag_pneumonia
FROM icd
GROUP BY hadm_id
"""

SQL["ed_icd_flags_sql"] = f"""
WITH hadms AS (SELECT x AS hadm_id FROM UNNEST(@hadms) AS x),
ed_dx AS (
  SELECT
    s.hadm_id,
    UPPER(REPLACE(d.icd_code, \".\", \"\")) AS code_norm
  FROM `{PHYS}.{ED}.diagnosis` d
  JOIN `{PHYS}.{ED}.edstays` s
    ON s.stay_id = d.stay_id
  WHERE s.hadm_id IN (SELECT hadm_id FROM hadms)
    AND (
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J43\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J44\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"G473\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"E662\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"I50\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"G12\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"G70\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"G71\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"F11\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"T40\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"F13\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J12\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J13\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J14\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J15\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J16\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J17\") OR
      STARTS_WITH(UPPER(REPLACE(d.icd_code, \".\", \"\")), \"J18\")
    )
)
SELECT
  hadm_id,
  MAX(IF(STARTS_WITH(code_norm, \"J43\") OR STARTS_WITH(code_norm, \"J44\"), 1, 0)) AS flag_copd,
  MAX(IF(STARTS_WITH(code_norm, \"G473\") OR STARTS_WITH(code_norm, \"E662\"), 1, 0)) AS flag_osa_ohs,
  MAX(IF(STARTS_WITH(code_norm, \"I50\"), 1, 0)) AS flag_chf,
  MAX(IF(STARTS_WITH(code_norm, \"G12\") OR STARTS_WITH(code_norm, \"G70\") OR STARTS_WITH(code_norm, \"G71\"), 1, 0)) AS flag_neuromuscular,
  MAX(IF(STARTS_WITH(code_norm, \"F11\") OR STARTS_WITH(code_norm, \"T40\") OR STARTS_WITH(code_norm, \"F13\"), 1, 0)) AS flag_opioid_substance,
  MAX(IF(STARTS_WITH(code_norm, \"J12\") OR STARTS_WITH(code_norm, \"J13\") OR STARTS_WITH(code_norm, \"J14\") OR STARTS_WITH(code_norm, \"J15\") OR STARTS_WITH(code_norm, \"J16\") OR STARTS_WITH(code_norm, \"J17\") OR STARTS_WITH(code_norm, \"J18\"), 1, 0)) AS flag_pneumonia
FROM ed_dx
GROUP BY hadm_id
"""

flag_hosp = run_sql_bq(sql("icd_flags_sql"), {"hadms": hadm_list})
flag_ed = run_sql_bq(sql("ed_icd_flags_sql"), {"hadms": hadm_list})

flag_hosp = flag_hosp.rename(columns={k: f"{k}_hosp" for k in FLAGS})
flag_ed = flag_ed.rename(columns={k: f"{k}_ed" for k in FLAGS})

flag_df = flag_hosp.merge(flag_ed, on="hadm_id", how="outer")
for k in FLAGS:
    hosp_col = f"{k}_hosp"
    ed_col = f"{k}_ed"
    if hosp_col not in flag_df.columns:
        flag_df[hosp_col] = 0
    if ed_col not in flag_df.columns:
        flag_df[ed_col] = 0
    flag_df[k] = ((flag_df[hosp_col].fillna(0).astype(int) == 1) | (flag_df[ed_col].fillna(0).astype(int) == 1)).astype(int)

# merge into admission-level and ED-stay-level frames (override if present)
for _df_name in ["df", "ed_df"]:
    if _df_name in globals():
        _df = globals()[_df_name]
        drop_cols = [c for c in flag_df.columns if c != "hadm_id" and c in _df.columns]
        if drop_cols:
            _df = _df.drop(columns=drop_cols)
        _df = _df.merge(flag_df, on="hadm_id", how="left")
        globals()[_df_name] = _df

# prevalence (combined flags)
for k in FLAGS:
    if k in ed_df.columns:
        print(k, int(ed_df[k].fillna(0).sum()))



flag_copd 2760
flag_osa_ohs 1891
flag_chf 4685
flag_neuromuscular 106
flag_opioid_substance 830
flag_pneumonia 2878


## Phase 8 — Timing phenotypes and derived bands

**Rationale:** Compute time-anchored hypercapnia/acidemia phenotypes for ED presentation vs later course.


In [39]:
# Timing phenotypes

anchor = "ed_intime_first" if "ed_intime_first" in ed_df.columns else "ed_intime"

ed_df["dt_first_qualifying_gas_hours"] = (ed_df["first_gas_time"] - ed_df[anchor]).dt.total_seconds() / 3600.0
ed_df["presenting_hypercapnia"] = (ed_df["dt_first_qualifying_gas_hours"] <= 6).astype("Int64")
ed_df["late_hypercapnia"] = (ed_df["dt_first_qualifying_gas_hours"] > 6).astype("Int64")

# NIV/IMV timing relative to first ED presentation
if "first_imv_time" in ed_df.columns:
    ed_df["dt_first_imv_hours"] = (ed_df["first_imv_time"] - ed_df[anchor]).dt.total_seconds() / 3600.0
if "first_niv_time" in ed_df.columns:
    ed_df["dt_first_niv_hours"] = (ed_df["first_niv_time"] - ed_df[anchor]).dt.total_seconds() / 3600.0

# Bands
bins_ph = [-1, 7.20, 7.30, 7.35, 99]
labels_ph = ["<7.20","7.20–7.29","7.30–7.34","≥7.35"]
ed_df["ph_band"] = pd.cut(ed_df["first_ph"], bins=bins_ph, labels=labels_ph)

bins_hco3 = [-1, 24, 30, 999]
labels_hco3 = ["<24","24–29","≥30"]
ed_df["hco3_band"] = pd.cut(ed_df["first_hco3"], bins=bins_hco3, labels=labels_hco3)

bins_lac = [-1, 2, 4, 999]
labels_lac = ["<2","2–4",">4"]
ed_df["lactate_band"] = pd.cut(ed_df["first_lactate"], bins=bins_lac, labels=labels_lac)



## QA / audits

**Rationale:** Validate joins, missingness, lab completeness, and produce reproducibility artifacts.


In [40]:
# --- QA checks (lightweight, deterministic)
STRICT_QA = False

# 1) Key uniqueness
if 'ed_df' in globals() and 'ed_stay_id' in ed_df.columns:
    try:
        assert_unique(ed_df, 'ed_stay_id', 'ed_df')
        print('QA: ed_stay_id unique OK')
    except Exception as e:
        print('QA FAIL:', e)
        if STRICT_QA:
            raise

# 2) Inclusion sanity: any_hypercap_icd==0 implies gas criteria met
if 'ed_df' in globals() and 'any_hypercap_icd' in ed_df.columns:
    gas_flag = ed_df.get('flag_any_gas_hypercapnia', pd.Series(0, index=ed_df.index)).fillna(0).astype(int)
    icd_flag = ed_df['any_hypercap_icd'].fillna(0).astype(int)
    viol = (icd_flag == 0) & (gas_flag == 0)
    n_viol = int(viol.sum())
    print('QA: ICD==0 & Gas==0 count:', n_viol)
    if STRICT_QA and n_viol > 0:
        raise AssertionError('Found rows without ICD and without gas criteria.')

# 3) Temporal sanity: first_gas_time >= ed_intime (allow small negative drift)
if 'ed_df' in globals() and 'first_gas_time' in ed_df.columns and 'ed_intime' in ed_df.columns:
    dt = (pd.to_datetime(ed_df['first_gas_time']) - pd.to_datetime(ed_df['ed_intime'])).dt.total_seconds() / 3600
    n_neg = int((dt < -1).sum())  # allow 1h clock drift
    print('QA: first_gas_time < ed_intime by >1h:', n_neg)
    if STRICT_QA and n_neg > 0:
        raise AssertionError('first_gas_time before ed_intime by >1h')

# 4) Range checks (report only)
if 'ed_df' in globals():
    ranges = {
        'first_ph': (6.8, 7.8),
        'first_pco2': (10, 200),
        'first_lactate': (0, 30),
        'creatinine': (0, 20),
    }
    rc = check_ranges(ed_df, ranges)
    if not rc.empty:
        print('QA range check (n out-of-range):')
        print(rc.to_string(index=False))



QA: ed_stay_id unique OK
QA: first_gas_time < ed_intime by >1h: 0
QA range check (n out-of-range):
          col  n_bad
     first_ph      7
   first_pco2      8
first_lactate      0


In [41]:
import json

# T1 — uniqueness and join explosion guard
if ed_df["ed_stay_id"].nunique() != len(ed_df):
    raise ValueError("ed_stay_id not unique after merges")

# T2 — missingness summary for new fields
new_fields = [c for c in TARGET_FIELDS if c in ed_df.columns]
miss = pd.DataFrame({
    "field": new_fields,
    "missing_n": [int(ed_df[c].isna().sum()) for c in new_fields],
    "missing_pct": [float(ed_df[c].isna().mean()) for c in new_fields],
})
print(miss.sort_values("missing_pct", ascending=False).head(30))

# T3 — lab capture completeness
pct_any_6h = float(p06["ed_stay_id"].nunique() / max(ed_df["ed_stay_id"].nunique(),1))
pct_any_24h = float(p24["ed_stay_id"].nunique() / max(ed_df["ed_stay_id"].nunique(),1))
print("% any gas panel 0–6h:", round(pct_any_6h*100,1))
print("% any gas panel 0–24h:", round(pct_any_24h*100,1))

# Use per-stay unknown rate if available
if "gas_source_unknown_rate" in ed_df.columns:
    source_unknown_rate = float(ed_df["gas_source_unknown_rate"].mean())
else:
    source_unknown_rate = 1.0
print("% source unknown (panel-level):", round(source_unknown_rate*100,1))

# T4 — QA summary artifact
qa_summary = {
    "ed_rows": int(len(ed_df)),
    "ed_unique": int(ed_df["ed_stay_id"].nunique()),
    "icu_link_rate": float(ed_df["icu_intime_first"].notna().mean()) if "icu_intime_first" in ed_df.columns else None,
    "pct_any_gas_0_6h": pct_any_6h,
    "pct_any_gas_0_24h": pct_any_24h,
    "source_unknown_rate": source_unknown_rate,
}
qa_path = WORK_DIR / "qa_summary.json"
qa_path.write_text(json.dumps(qa_summary, indent=2))
print("Wrote:", qa_path)



                    field  missing_n  missing_pct
76  weight_closest_pre_ed      17449     1.000000
4      bmi_closest_pre_ed      17449     1.000000
54  height_closest_pre_ed      17449     1.000000
53              hco3_band      16866     0.966588
38             first_hco3      16866     0.966588
16        ed_first_rhythm      16748     0.959826
5               deathtime      15131     0.867156
69          max_pco2_0_6h      13399     0.767895
71            min_ph_0_6h      13278     0.760961
10     dt_first_niv_hours      12743     0.730300
49         flag_pneumonia       9526     0.545934
44               flag_chf       9526     0.545934
45              flag_copd       9526     0.545934
46     flag_neuromuscular       9526     0.545934
47  flag_opioid_substance       9526     0.545934
48           flag_osa_ohs       9526     0.545934
9      dt_first_imv_hours       7150     0.409766
63           lactate_band       6141     0.351940
39          first_lactate       6141     0.351940


## Outputs (ED-stay cohort + long tables)

**Rationale:** Persist ED-stay analytic datasets and supporting long tables.


In [42]:
# Save outputs
cohort_path = DATA_DIR / "cohort_ed_stay.parquet"
ed_df.to_parquet(cohort_path, index=False)
print("Wrote:", cohort_path)

ed_vitals_long.to_parquet(DATA_DIR / "ed_vitals_long.parquet", index=False)
labs_long.to_parquet(DATA_DIR / "labs_long.parquet", index=False)
panel.to_parquet(DATA_DIR / "gas_panels.parquet", index=False)





Wrote: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/cohort_ed_stay.parquet


In [43]:
# Also export to Excel (tabular dataset)
from datetime import datetime

out_dir = DATA_DIR
out_dir.mkdir(parents=True, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

xlsx_path = out_dir / f"mimic_hypercap_EXT_EDstay_bq_gas_{timestamp}.xlsx"

with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xw:
    ed_df.to_excel(xw, sheet_name="cohort_ed_stay", index=False)
    # Optional: include QA tables if present
    try:
        miss.to_excel(xw, sheet_name="missingness", index=False)
    except Exception:
        pass

print("Saved:", xlsx_path)



Saved: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/mimic_hypercap_EXT_EDstay_bq_gas_20260204_211114.xlsx


In [44]:
# Final Excel outputs requested: all encounters + ED chief-complaint only
from datetime import datetime

out_dir = DATA_DIR
out_dir.mkdir(parents=True, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 1) All encounters meeting inclusion criteria (admission-level), with ED linkage flags
ed_link = (
    ed_df.groupby("hadm_id", as_index=False)
         .agg(ed_stay_id_first=("ed_stay_id", "first"), n_ed_stays=("ed_stay_id", "nunique"))
)
all_encounters = df.merge(ed_link, on="hadm_id", how="left")
all_encounters["has_ed_encounter"] = all_encounters["n_ed_stays"].fillna(0).astype(int).gt(0).astype(int)
all_encounters["encounter_source"] = all_encounters["has_ed_encounter"].map({1: "ED-linked", 0: "Inpatient-only"})

all_path = out_dir / f"mimic_hypercap_EXT_all_encounters_bq_{timestamp}.xlsx"
with pd.ExcelWriter(all_path, engine="openpyxl") as xw:
    all_encounters.to_excel(xw, sheet_name="all_encounters", index=False)
print("Saved:", all_path)

# 2) ED chief-complaint-only (ED-stay level)
if "ed_triage_cc" not in ed_df.columns:
    raise KeyError("Column 'ed_triage_cc' not found in ed_df. Ensure ED triage merge ran.")

mask_cc = ed_df["ed_triage_cc"].notna() & (ed_df["ed_triage_cc"].astype(str).str.strip() != "")
ed_cc_only = ed_df.loc[mask_cc].copy()

cc_path = out_dir / f"mimic_hypercap_EXT_EDcc_only_edstay_bq_{timestamp}.xlsx"
with pd.ExcelWriter(cc_path, engine="openpyxl") as xw:
    ed_cc_only.to_excel(xw, sheet_name="ed_cc_only", index=False)
print("Saved:", cc_path)



Saved: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/mimic_hypercap_EXT_all_encounters_bq_20260204_211130.xlsx
Saved: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/mimic_hypercap_EXT_EDcc_only_edstay_bq_20260204_211130.xlsx


In [45]:
# Final CC output (cohort-only; excludes NLP-derived columns)
from datetime import datetime

# Ensure cc-only frames exist
if "df_cc" not in globals():
    raise NameError("df_cc not found. Run the ED-CC-only export cell first.")

if "ed_cc_only" not in globals():
    if "ed_df" not in globals():
        raise NameError("ed_df not found. Run the ED-stay build cells first.")
    if "ed_triage_cc" not in ed_df.columns:
        raise KeyError("ed_triage_cc missing in ed_df; cannot build ed_cc_only.")
    _mask_cc = ed_df["ed_triage_cc"].notna() & (ed_df["ed_triage_cc"].astype(str).str.strip() != "")
    ed_cc_only = ed_df.loc[_mask_cc].copy()

# Add ED-stay columns (dedup to earliest ED stay per hadm_id)
ed_tmp = ed_cc_only.copy()
if "ed_intime" in ed_tmp.columns:
    ed_tmp = ed_tmp.sort_values(["hadm_id", "ed_intime", "ed_stay_id"], na_position="last")
else:
    ed_tmp = ed_tmp.sort_values(["hadm_id", "ed_stay_id"], na_position="last")
ed_first = ed_tmp.drop_duplicates("hadm_id", keep="first")

add_ed_cols = [c for c in ed_first.columns if c not in df_cc.columns]

final_cc = df_cc.merge(ed_first[["hadm_id"] + add_ed_cols], on="hadm_id", how="left")

out_date = datetime.now().strftime("%Y-%m-%d")
out_path = DATA_DIR / f"{out_date} MIMICIV all with CC.xlsx"
final_cc.to_excel(out_path, index=False)

print("Saved:", out_path)
print("Base rows:", len(df_cc), "| Added from ED-stay:", len(add_ed_cols))
print("Total columns:", len(final_cc.columns))



Saved: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/2026-02-04 MIMICIV all with CC.xlsx
Base rows: 17424 | Added from ED-stay: 68
Total columns: 157


In [46]:
# Data dictionary (OSF-style) for final cohort output
from datetime import datetime
import numpy as np

# Choose target dataset for dictionary
if "final_cc" in globals():
    dd_target = final_cc
elif "df_cc" in globals():
    dd_target = df_cc
elif "df" in globals():
    dd_target = df
else:
    raise NameError("No cohort dataframe found for data dictionary (expected final_cc/df_cc/df).")

# Helper functions

def _readable(name: str) -> str:
    return name.replace("_", " ").strip().title()


def _infer_units(col: str) -> str:
    if col.endswith("_hours"):
        return "hours"
    if col.endswith("_time") or col.endswith("_intime") or col.endswith("_outtime") or col.endswith("_date"):
        return "datetime"
    if col.startswith("flag_") or col.endswith("_flag") or col.endswith("_before_imv"):
        return "binary (0/1)"
    if col.endswith("_paco2") or col.endswith("_pco2"):
        return "mmHg"
    if col.endswith("_ph"):
        return "pH"
    if col.endswith("_lactate"):
        return "mmol/L"
    return ""


def _allowed_values(series):
    s = series.dropna()
    if s.empty:
        return ""
    if s.dtype.kind in "biu":
        vals = sorted(map(int, s.unique()))
        if len(vals) <= 15:
            return ", ".join(map(str, vals))
        return f"{min(vals)}–{max(vals)}"
    if s.dtype.kind in "f":
        return f"{np.nanmin(s):.3g}–{np.nanmax(s):.3g}"
    # object/categorical
    vals = sorted(map(str, s.unique()))
    return ", ".join(vals[:15]) + (" …" if len(vals) > 15 else "")


META = {
    # IDs
    "subject_id": {
        "label": "Subject identifier",
        "definition": "Identifier for a unique patient in MIMIC-IV.",
        "source": "mimiciv_hosp.patients",
        "derivation": "as recorded",
    },
    "hadm_id": {
        "label": "Hospital admission identifier",
        "definition": "Identifier for a hospital admission.",
        "source": "mimiciv_hosp.admissions",
        "derivation": "as recorded",
    },
    "ed_stay_id": {
        "label": "ED stay identifier",
        "definition": "Identifier for an ED stay.",
        "source": "mimiciv_ed.edstays",
        "derivation": "as recorded",
    },
    "ed_intime": {
        "label": "ED arrival time",
        "definition": "ED arrival time for this ED stay.",
        "source": "mimiciv_ed.edstays",
        "derivation": "as recorded",
        "units": "datetime",
    },
    "ed_outtime": {
        "label": "ED departure time",
        "definition": "ED departure time for this ED stay.",
        "source": "mimiciv_ed.edstays",
        "derivation": "as recorded",
        "units": "datetime",
    },
    "ed_intime_first": {
        "label": "First ED arrival time (admission)",
        "definition": "Earliest ED arrival time among ED stays linked to the admission.",
        "source": "mimiciv_ed.edstays",
        "derivation": "min(edstays.intime) per hadm_id",
        "units": "datetime",
    },
    # Demographics / triage
    "ed_gender": {
        "label": "ED gender",
        "definition": "Gender as recorded in ED stay table.",
        "source": "mimiciv_ed.edstays",
        "derivation": "as recorded",
    },
    "ed_race": {
        "label": "ED race",
        "definition": "Race as recorded in ED stay table.",
        "source": "mimiciv_ed.edstays",
        "derivation": "as recorded",
    },
    "hosp_race": {
        "label": "Hospital race",
        "definition": "Race as recorded in admissions table.",
        "source": "mimiciv_hosp.admissions",
        "derivation": "as recorded",
    },
    "ed_triage_cc": {
        "label": "ED chief complaint",
        "definition": "Chief complaint recorded at ED triage.",
        "source": "mimiciv_ed.triage",
        "derivation": "as recorded",
    },
    "ed_triage_acuity": {
        "label": "ED triage acuity",
        "definition": "Triage acuity level recorded at ED presentation.",
        "source": "mimiciv_ed.triage",
        "derivation": "as recorded",
    },
    # Outcomes
    "hospital_expire_flag": {
        "label": "Hospital expire flag",
        "definition": "Hospital mortality indicator from admissions table.",
        "source": "mimiciv_hosp.admissions",
        "derivation": "as recorded",
        "units": "binary (0/1)",
    },
    "in_hospital_death": {
        "label": "In-hospital death",
        "definition": "Indicator for in-hospital death during admission.",
        "source": "mimiciv_hosp.admissions",
        "derivation": "(hospital_expire_flag == 1) OR (deathtime not null)",
        "units": "binary (0/1)",
    },
    # Ventilation
    "imv_flag": {
        "label": "IMV flag",
        "definition": "Indicator of invasive mechanical ventilation during admission.",
        "source": "ICD procedures + ICU chartevents",
        "derivation": "max(IMV ICD flag, IMV chart flag)",
        "units": "binary (0/1)",
    },
    "niv_flag": {
        "label": "NIV flag",
        "definition": "Indicator of noninvasive ventilation during admission.",
        "source": "ICD procedures + ICU chartevents",
        "derivation": "max(NIV ICD flag, NIV chart flag)",
        "units": "binary (0/1)",
    },
    "first_imv_time": {
        "label": "First IMV time",
        "definition": "Earliest charted time of IMV in ICU chartevents (if present).",
        "source": "mimiciv_icu.chartevents",
        "derivation": "min charttime among IMV charted events",
        "units": "datetime",
    },
    "first_niv_time": {
        "label": "First NIV time",
        "definition": "Earliest charted time of NIV in ICU chartevents (if present).",
        "source": "mimiciv_icu.chartevents",
        "derivation": "min charttime among NIV charted events",
        "units": "datetime",
    },
    "dt_first_imv_hours": {
        "label": "Hours to first IMV",
        "definition": "Hours from first ED arrival to first IMV time.",
        "source": "Derived",
        "derivation": "(first_imv_time - ed_intime_first) in hours",
        "units": "hours",
    },
    "dt_first_niv_hours": {
        "label": "Hours to first NIV",
        "definition": "Hours from first ED arrival to first NIV time.",
        "source": "Derived",
        "derivation": "(first_niv_time - ed_intime_first) in hours",
        "units": "hours",
    },
    "abg_before_imv": {
        "label": "ABG before IMV",
        "definition": "Indicator that first ABG time precedes first IMV time.",
        "source": "Derived",
        "derivation": "first_abg_time < first_imv_time",
        "units": "binary (0/1)",
    },
    "vbg_before_imv": {
        "label": "VBG before IMV",
        "definition": "Indicator that first VBG time precedes first IMV time.",
        "source": "Derived",
        "derivation": "first_vbg_time < first_imv_time",
        "units": "binary (0/1)",
    },
    # Gas/ABG/VBG
    "abg_hypercap_threshold": {
        "label": "ABG hypercapnia threshold met",
        "definition": "Indicator that any arterial pCO2 ≥ 45 mmHg in hosp/ICU pCO2 extraction.",
        "source": "mimiciv_hosp.labevents + mimiciv_icu.chartevents",
        "derivation": "max(arterial pCO2 ≥ 45) per hadm_id",
        "units": "binary (0/1)",
    },
    "vbg_hypercap_threshold": {
        "label": "VBG hypercapnia threshold met",
        "definition": "Indicator that any venous pCO2 ≥ 50 mmHg in hosp/ICU pCO2 extraction.",
        "source": "mimiciv_hosp.labevents + mimiciv_icu.chartevents",
        "derivation": "max(venous pCO2 ≥ 50) per hadm_id",
        "units": "binary (0/1)",
    },
    "first_gas_time": {
        "label": "First gas panel time",
        "definition": "Earliest gas panel time in the ED 0–24h window.",
        "source": "mimiciv_hosp.labevents (gas panels)",
        "derivation": "min panel_time within ED 0–24h window",
        "units": "datetime",
    },
    "dt_first_qualifying_gas_hours": {
        "label": "Hours to first gas panel",
        "definition": "Hours from first ED arrival to first gas panel time.",
        "source": "Derived",
        "derivation": "(first_gas_time - ed_intime_first) in hours",
        "units": "hours",
    },
    "presenting_hypercapnia": {
        "label": "Presenting (≤6h) gas timing",
        "definition": "Indicator that first gas panel occurred within 6h of first ED arrival.",
        "source": "Derived",
        "derivation": "dt_first_qualifying_gas_hours ≤ 6",
        "units": "binary (0/1)",
    },
    "late_hypercapnia": {
        "label": "Late (>6h) gas timing",
        "definition": "Indicator that first gas panel occurred >6h after first ED arrival.",
        "source": "Derived",
        "derivation": "dt_first_qualifying_gas_hours > 6",
        "units": "binary (0/1)",
    },
    "gas_source_unknown_rate": {
        "label": "Gas source unknown rate",
        "definition": "Proportion of gas panels with unknown arterial/venous source per ED stay.",
        "source": "Derived from gas panel source inference",
        "derivation": "mean(source == 'unknown') per ed_stay_id",
        "units": "proportion",
    },
    # Comorbidities
    "flag_copd": {
        "label": "COPD/emphysema flag",
        "definition": "Indicator for COPD/emphysema ICD codes in ED or hospital diagnoses.",
        "source": "mimiciv_hosp.diagnoses_icd + mimiciv_ed.diagnosis",
        "derivation": "ICD prefixes J43/J44 (ED OR hospital)",
        "units": "binary (0/1)",
    },
    "flag_osa_ohs": {
        "label": "OSA/OHS flag",
        "definition": "Indicator for OSA/OHS ICD codes in ED or hospital diagnoses.",
        "source": "mimiciv_hosp.diagnoses_icd + mimiciv_ed.diagnosis",
        "derivation": "ICD prefixes G473/E662 (ED OR hospital)",
        "units": "binary (0/1)",
    },
    "flag_chf": {
        "label": "CHF flag",
        "definition": "Indicator for CHF ICD codes in ED or hospital diagnoses.",
        "source": "mimiciv_hosp.diagnoses_icd + mimiciv_ed.diagnosis",
        "derivation": "ICD prefix I50 (ED OR hospital)",
        "units": "binary (0/1)",
    },
    "flag_neuromuscular": {
        "label": "Neuromuscular flag",
        "definition": "Indicator for neuromuscular ICD codes in ED or hospital diagnoses.",
        "source": "mimiciv_hosp.diagnoses_icd + mimiciv_ed.diagnosis",
        "derivation": "ICD prefixes G12/G70/G71 (ED OR hospital)",
        "units": "binary (0/1)",
    },
    "flag_opioid_substance": {
        "label": "Opioid/substance flag",
        "definition": "Indicator for opioid/substance ICD codes in ED or hospital diagnoses.",
        "source": "mimiciv_hosp.diagnoses_icd + mimiciv_ed.diagnosis",
        "derivation": "ICD prefixes F11/T40/F13 (ED OR hospital)",
        "units": "binary (0/1)",
    },
    "flag_pneumonia": {
        "label": "Pneumonia flag",
        "definition": "Indicator for pneumonia ICD codes in ED or hospital diagnoses.",
        "source": "mimiciv_hosp.diagnoses_icd + mimiciv_ed.diagnosis",
        "derivation": "ICD prefixes J12–J18 (ED OR hospital)",
        "units": "binary (0/1)",
    },
}

rows = []
for col in dd_target.columns:
    info = META.get(col, {})
    series = dd_target[col]
    rows.append({
        "variable": col,
        "label": info.get("label", _readable(col)),
        "units": info.get("units", _infer_units(col)),
        "allowed_values": info.get("allowed_values", _allowed_values(series)),
        "definition": info.get("definition", "UNCONFIRMED"),
        "synonyms": info.get("synonyms", ""),
        "description": info.get("description", ""),
        "source": info.get("source", "UNCONFIRMED"),
        "derivation": info.get("derivation", "UNCONFIRMED"),
        "dtype": str(series.dtype),
        "example_value": series.dropna().iloc[0] if series.dropna().shape[0] else None,
    })

data_dict = pd.DataFrame(rows)

out_date = datetime.now().strftime("%Y-%m-%d")
out_xlsx = DATA_DIR / f"{out_date} MIMICIV all with CC_data_dictionary.xlsx"
out_csv = DATA_DIR / f"{out_date} MIMICIV all with CC_data_dictionary.csv"

data_dict.to_excel(out_xlsx, index=False)
data_dict.to_csv(out_csv, index=False)

print("Saved data dictionary:", out_xlsx)
print("Saved data dictionary:", out_csv)
print("Rows:", len(data_dict))



Saved data dictionary: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/2026-02-04 MIMICIV all with CC_data_dictionary.xlsx
Saved data dictionary: /Users/blocke/Box Sync/Residency Personal Files/Scholarly Work/Locke Research Projects/Hypercap-CC-NLP/MIMIC tabular data/2026-02-04 MIMICIV all with CC_data_dictionary.csv
Rows: 157
